In [1]:
# qc_forecast_playbook.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb (replacement) units
# to pre-stock for the next H days, using MVP (moving average) and PRO
# (lifecycle bucket hazard) approaches.
#
# Inputs (CSV with headers; date columns can be YYYY-MM-DD):
#   products.csv: product_id, product_name, category_id, launch_date, price (optional)
#   stores.csv:   store_id, store_name, city, country
#   sales.csv:    sale_id, sale_date, store_id, product_id, quantity
#   warranty.csv: claim_id, claim_date, sale_id, repair_status (optional)
#
# Outputs (CSV):
#   qc_hotspots.csv            - recent 180d risk ranking by claim probability (store×product)
#   qc_mvp_forecast.csv        - MVP forecast (moving average + safety stock)
#   qc_pro_forecast.csv        - PRO forecast (bucket hazard × installed base)
#   qc_final_forecast.csv      - blended final recommendation with dates
#
# Usage:
#   1) Edit the PATHS dict in __main__ to point to your CSVs.
#   2) Adjust parameters in PARAMS as needed.
#   3) Run:  python qc_forecast_playbook.py
#
# Requires: pandas, numpy (no external ML libs).
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict, Tuple, Optional

# ---------------------------
# Parameters
# ---------------------------
DEFAULT_PARAMS = {
    "HORIZON_DAYS": 30,          # planning horizon window
    "K_MONTHS_MA": 3,            # moving average window (months)
    "LEAD_TIME_DAYS": 14,        # replenishment lead time
    "SERVICE_Z": 1.65,           # 95% service level (z=1.65), 99% -> 2.33
    "LOOKBACK_DAYS_HOTSPOT": 365,# hotspot ranking lookback
    "BUCKET_DAYS": [0, 180, 360],# lifecycle buckets: [0-180], [181-360], 361+
    "BLEND_WEIGHT_PRO": 0.7,     # final blend: 70% PRO + 30% MVP (tune as needed)
    "MIN_HISTORY_CLAIMS": 30     # if product total claims < this, treat as sparse history
}


# ---------------------------
# Utilities
# ---------------------------
def _to_date(s):
    """Convert series to date, handling errors gracefully"""
    return pd.to_datetime(s, errors="coerce").dt.date

def _days_between(a, b):
    """Calculate days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def _safe_div(a, b):
    """Safe division avoiding division by zero"""
    return np.where(b == 0, 0.0, a / b)

def _ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

def _validate_file_exists(filepath: str) -> None:
    """Validate that file exists"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

def _validate_required_columns(df: pd.DataFrame, required_cols: list, filename: str) -> None:
    """Validate that required columns exist in dataframe"""
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in {filename}: {missing}")


# ---------------------------
# Loading / Cleaning
# ---------------------------
def load_data(paths: Dict[str, str]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, datetime.date]:
    """Load and validate data from CSV files"""
    
    # Validate files exist
    for name, path in paths.items():
        _validate_file_exists(path)
    
    try:
        # Load data
        products = pd.read_csv(paths["products"])
        stores = pd.read_csv(paths["stores"])
        sales = pd.read_csv(paths["sales"])
        warranty = pd.read_csv(paths["warranty"])
        
        # Validate required columns
        _validate_required_columns(products, ["product_id", "product_name", "category_id"], "products.csv")
        _validate_required_columns(stores, ["store_id", "store_name", "city", "country"], "stores.csv")
        _validate_required_columns(sales, ["sale_id", "sale_date", "store_id", "product_id", "quantity"], "sales.csv")
        _validate_required_columns(warranty, ["claim_id", "claim_date", "sale_id"], "warranty.csv")
        
        # Convert dates
        if "launch_date" in products.columns:
            products["launch_date"] = _to_date(products["launch_date"])
        sales["sale_date"] = _to_date(sales["sale_date"])
        warranty["claim_date"] = _to_date(warranty["claim_date"])
        
        # Basic cleaning
        sales = sales[sales["quantity"] > 0].copy()
        
        # Remove rows with invalid dates
        sales = sales.dropna(subset=["sale_date"])
        warranty = warranty.dropna(subset=["claim_date"])
        
        # Deduplicate
        sales = sales.drop_duplicates(subset=["sale_id"])
        warranty = warranty.drop_duplicates(subset=["claim_id"])
        
        # Calculate TODAY from actual data
        today = max(
            sales["sale_date"].max() if not sales.empty else datetime.now().date(),
            warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
        )
        
        print(f"Data loaded successfully. TODAY calculated as: {today}")
        print(f"Sales records: {len(sales)}, Warranty claims: {len(warranty)}")
        
        return products, stores, sales, warranty, today
        
    except Exception as e:
        raise ValueError(f"Error loading data: {str(e)}")


# ---------------------------
# Joins
# ---------------------------
def join_sales_warranty(products, stores, sales, warranty):
    """Join warranty claims with sales and product/store info"""
    ws = (warranty
          .merge(sales[["sale_id", "sale_date", "store_id", "product_id", "quantity"]], 
                 on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                 on="product_id", how="left")
          .merge(stores[["store_id", "store_name", "city", "country"]], 
                 on="store_id", how="left")
         )
    return ws


# ---------------------------
# Hotspot ranking (recent 180d)
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, lookback_days=180):
    """Build hotspot ranking based on recent claim probability"""
    cutoff = today - timedelta(days=lookback_days)
    
    # Recent sales and warranty claims
    s_recent = sales[sales["sale_date"] >= cutoff].copy()
    w_recent = warranty[warranty["claim_date"] >= cutoff].copy()
    
    if s_recent.empty:
        print("Warning: No recent sales data found for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales
    base = (s_recent
            .merge(products[["product_id", "product_name"]], on="product_id", how="left")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Aggregate claims
    claims = pd.DataFrame()
    if not w_recent.empty:
        claims = (w_recent
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .merge(products[["product_id", "product_name"]], on="product_id", how="left")
                  .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
                  .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
                  .agg(claims=("claim_id", "count"))
                  .reset_index())
    
    # Merge and calculate probabilities
    if claims.empty:
        base["claims"] = 0
        df = base
    else:
        df = base.merge(claims, on=["country", "store_name", "product_id", "product_name"], how="left")
        df["claims"] = df["claims"].fillna(0).astype(int)
    
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    df = df.sort_values(["risk_rank", "country", "store_name", "product_name"])
    
    return df


# ---------------------------
# Installed base & monthly claims
# ---------------------------
def build_installed_base(stores, products, sales, warranty):
    """Calculate installed base (sold - claimed) per store×product"""
    # Cumulative sales
    sold = (sales
            .groupby(["store_id", "product_id"], as_index=False)
            .agg(units_sold=("quantity", "sum")))
    
    # Cumulative claims
    claims = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    if not warranty.empty:
        claims = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .groupby(["store_id", "product_id"], as_index=False)
                  .agg(units_claimed=("claim_id", "count")))
    
    # Merge and calculate installed base
    base = sold.merge(claims, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0).astype(int)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                   on="product_id", how="left"))
    
    return base[["country", "store_name", "product_id", "product_name", 
                 "category_id", "launch_date", "installed_base"]]


def build_claims_monthly(products, stores, sales, warranty):
    """Build monthly claims aggregation"""
    if warranty.empty:
        return pd.DataFrame()
    
    ws = join_sales_warranty(products, stores, sales, warranty)
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    grp = (ws.groupby(["country", "store_name", "product_id", "product_name", "month"], dropna=False)
             .agg(claims=("claim_id", "count")).reset_index())
    
    return grp


# ---------------------------
# MVP forecast (moving average + safety)
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Generate MVP forecast using moving average approach"""
    horizon_days = params["HORIZON_DAYS"]
    k_months = params["K_MONTHS_MA"]
    z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    
    cm = build_claims_monthly(products, stores, sales, warranty)
    
    if cm.empty:
        print("Warning: No claims data available for MVP forecast")
        return pd.DataFrame()
    
    # Keep last k months for stats
    cutoff_period = pd.Timestamp(today).to_period("M") - k_months
    cm["month"] = pd.to_datetime(cm["month"].astype(str))
    cm["month_period"] = cm["month"].dt.to_period("M")
    recent = cm[cm["month_period"] >= cutoff_period]
    
    if recent.empty:
        print("Warning: No recent claims data for MVP forecast")
        return pd.DataFrame()
    
    # Calculate statistics
    stats = (recent
             .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
             .agg(ma_k="mean", sd_k="std").reset_index())
    stats["sd_k"] = stats["sd_k"].fillna(0.0)
    
    # Convert monthly to horizon days
    stats["demand_horizon"] = _ensure_nonneg_int(np.ceil(stats["ma_k"] * (horizon_days / 30.0)))
    stats["safety_stock"] = _ensure_nonneg_int(np.ceil(z * stats["sd_k"] * np.sqrt(lead_time / 30.0)))
    stats["units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    stats["horizon_end"] = horizon_end
    stats["latest_stock_date"] = latest_stock_date
    
    # Rename columns and sort
    out = stats.rename(columns={
        "ma_k": "mvp_ma_month",
        "sd_k": "mvp_sd_month",
        "demand_horizon": "mvp_demand_horizon",
        "safety_stock": "mvp_safety",
        "units_to_prestore": "mvp_units_to_prestore"
    })
    
    out = out.sort_values(["mvp_units_to_prestore", "country", "store_name", "product_name"], 
                         ascending=[False, True, True, True])
    
    return out


# ---------------------------
# PRO forecast (bucket hazard × installed base)
# ---------------------------
def build_product_claim_prob(products, stores, sales, warranty):
    """Calculate product-level claim probability"""
    # Total sales per product
    sold = (sales.groupby(["product_id"], as_index=False)
                 .agg(units_sold=("quantity", "sum")))
    
    # Total claims per product
    claims = pd.DataFrame(columns=["product_id", "claims"])
    if not warranty.empty:
        claims = (warranty.merge(sales[["sale_id", "product_id"]], on="sale_id", how="left")
                         .groupby(["product_id"], as_index=False)
                         .agg(claims=("claim_id", "count")))
    
    # Merge and calculate probability
    df = sold.merge(claims, on="product_id", how="left")
    df["claims"] = df["claims"].fillna(0).astype(int)
    df["claim_prob"] = _safe_div(df["claims"], df["units_sold"])
    
    # Add product info
    df = df.merge(products[["product_id", "product_name", "category_id"]], on="product_id", how="left")
    
    return df[["product_id", "product_name", "category_id", "claim_prob", "claims", "units_sold"]]


def build_lifecycle_bucket_claims(products, sales, warranty, params):
    """Count claims by lifecycle bucket per product"""
    if warranty.empty:
        return pd.DataFrame()
    
    buckets = params["BUCKET_DAYS"]
    ws = (warranty
          .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "launch_date"]], on="product_id", how="left"))
    
    ws["age_days"] = _days_between(ws["claim_date"], ws["launch_date"])
    
    def bucket_label(d):
        if pd.isna(d):
            return "unknown"
        d = int(d)
        if d < 0:
            return "prelaunch"
        if d <= buckets[1]:    # 0-180
            return "0-6"
        if d <= buckets[2]:    # 181-360
            return "7-12"
        return "13+"
    
    ws["bucket"] = ws["age_days"].apply(bucket_label)
    c = (ws.groupby(["product_id", "bucket"], dropna=False)
           .agg(claims=("claim_id", "count")).reset_index())
    
    return c


def pro_forecast(products, stores, sales, warranty, today, params):
    """Generate PRO forecast using lifecycle bucket approach"""
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"]
    z = params["SERVICE_Z"]
    buckets = params["BUCKET_DAYS"]
    min_hist = params["MIN_HISTORY_CLAIMS"]
    k_months = params["K_MONTHS_MA"]
    
    # Get installed base
    base = build_installed_base(stores, products, sales, warranty)
    
    if base.empty:
        print("Warning: No installed base data for PRO forecast")
        return pd.DataFrame()
    
    # Product-level probabilities
    prod_prob = build_product_claim_prob(products, stores, sales, warranty)
    bucket_claims = build_lifecycle_bucket_claims(products, sales, warranty, params)
    
    # Handle empty bucket claims
    if bucket_claims.empty:
        # Create dummy bucket data
        unique_products = prod_prob["product_id"].unique()
        bucket_data = []
        for pid in unique_products:
            for bucket in ["0-6", "7-12", "13+"]:
                bucket_data.append({"product_id": pid, "bucket": bucket, "claims": 0})
        bucket_claims = pd.DataFrame(bucket_data)
    
    # Pivot bucket claims
    pivot = bucket_claims.pivot_table(index="product_id", columns="bucket", values="claims", 
                                     aggfunc="sum", fill_value=0).reset_index()
    
    # Merge with product probabilities
    prod = prod_prob.merge(pivot, on="product_id", how="left").fillna(0)
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in prod.columns:
            prod[b] = 0
    
    prod["total_claims_hist"] = prod[["0-6", "7-12", "13+"]].sum(axis=1)
    
    # Calculate monthly rates per bucket
    def compute_monthly_rates(row):
        total = row["total_claims_hist"]
        cp = row["claim_prob"]
        
        # Calculate shares
        if total > 0:
            s0 = row["0-6"] / total
            s1 = row["7-12"] / total
            s2 = row["13+"] / total
        else:
            s0 = s1 = s2 = 1/3.0
        
        # Monthly rates
        r0 = cp * s0 / 6.0    # 0-6 months
        r1 = cp * s1 / 6.0    # 7-12 months
        r2 = cp * s2 / 999.0  # 13+ months (very low rate)
        
        return pd.Series({"rate_0_6": r0, "rate_7_12": r1, "rate_13p": r2})
    
    rates = prod.apply(compute_monthly_rates, axis=1)
    prod = pd.concat([prod, rates], axis=1)
    
    # Determine current age bucket for each store×product
    base["age_days_now"] = _days_between(today, base["launch_date"])
    
    def current_bucket(d):
        if pd.isna(d):
            return "13+"
        d = int(d)
        if d <= buckets[1]:
            return "0-6"
        if d <= buckets[2]:
            return "7-12"
        return "13+"
    
    base["current_bucket"] = base["age_days_now"].apply(current_bucket)
    
    # Aggregate exposure by bucket
    exposure = (base
                .groupby(["country", "store_name", "product_id", "product_name", "current_bucket"], 
                        dropna=False)
                .agg(units=("installed_base", "sum"))
                .reset_index())
    
    # Pivot exposure
    expo_pivot = (exposure
                  .pivot_table(index=["country", "store_name", "product_id", "product_name"],
                              columns="current_bucket", values="units", aggfunc="sum", fill_value=0)
                  .reset_index())
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in expo_pivot.columns:
            expo_pivot[b] = 0
    
    # Merge with rates
    expo_pivot = expo_pivot.merge(prod[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims_hist"]], 
                                 on="product_id", how="left")
    
    # Handle sparse history
    sparse_mask = expo_pivot["total_claims_hist"].fillna(0) < min_hist
    for col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        expo_pivot.loc[sparse_mask, col] = expo_pivot.loc[sparse_mask, col] * 1.25
    
    # Calculate expected claims
    factor = horizon_days / 30.0
    expo_pivot["expected_claims"] = (
        expo_pivot["0-6"] * (expo_pivot["rate_0_6"] * factor) +
        expo_pivot["7-12"] * (expo_pivot["rate_7_12"] * factor) +
        expo_pivot["13+"] * (expo_pivot["rate_13p"] * factor)
    )
    
    # Safety stock calculation
    cm = build_claims_monthly(products, stores, sales, warranty)
    if not cm.empty:
        cutoff_period = pd.Timestamp(today).to_period("M") - k_months
        cm["month"] = pd.to_datetime(cm["month"].astype(str))
        cm["month_period"] = cm["month"].dt.to_period("M")
        recent = cm[cm["month_period"] >= cutoff_period]
        
        sd = (recent.groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
                    .std().reset_index().rename(columns={"claims": "sd_month"}))
        sd["sd_month"] = sd["sd_month"].fillna(0.0)
    else:
        sd = pd.DataFrame(columns=["country", "store_name", "product_id", "product_name", "sd_month"])
    
    # Merge safety stock data
    out = expo_pivot.merge(sd, on=["country", "store_name", "product_id", "product_name"], how="left")
    out["sd_month"] = out["sd_month"].fillna(0.0)
    
    # Calculate final values
    out["safety_stock"] = _ensure_nonneg_int(np.ceil(z * out["sd_month"] * np.sqrt(lead_time / 30.0)))
    out["expected_claims"] = _ensure_nonneg_int(np.ceil(out["expected_claims"]))
    out["pro_units_to_prestore"] = out["expected_claims"] + out["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    out["horizon_end"] = horizon_end
    out["latest_stock_date"] = latest_stock_date
    
    return out[["country", "store_name", "product_id", "product_name",
                "expected_claims", "safety_stock", "pro_units_to_prestore",
                "horizon_end", "latest_stock_date"]]


# ---------------------------
# Final blend & save
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save results"""
    w = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty dataframes
    if mvp.empty and pro.empty:
        print("Warning: Both MVP and PRO forecasts are empty")
        combined = pd.DataFrame()
    elif mvp.empty:
        combined = pro.copy()
        combined["final_units_to_prestore"] = combined["pro_units_to_prestore"]
    elif pro.empty:
        combined = mvp.copy()
        combined["final_units_to_prestore"] = combined["mvp_units_to_prestore"]
    else:
        # Merge forecasts
        combined = pd.merge(pro, mvp, on=keys, how="outer")
        
        # Fill NaNs
        for col in ["pro_units_to_prestore", "mvp_units_to_prestore", "expected_claims", 
                   "mvp_demand_horizon", "mvp_safety"]:
            if col in combined.columns:
                combined[col] = combined[col].fillna(0)
        
        # Blend forecasts
        combined["final_units_to_prestore"] = _ensure_nonneg_int(
            np.ceil(w * combined["pro_units_to_prestore"] + (1-w) * combined["mvp_units_to_prestore"])
        )
        
        # Handle date columns
        if "horizon_end_x" in combined.columns and "horizon_end_y" in combined.columns:
            combined["horizon_end"] = combined["horizon_end_x"].fillna(combined["horizon_end_y"])
            combined["latest_stock_date"] = combined["latest_stock_date_x"].fillna(combined["latest_stock_date_y"])
            drop_cols = [c for c in combined.columns if c.endswith("_x") or c.endswith("_y")]
            combined = combined.drop(columns=drop_cols)
    
    # Add hotspot data if available
    if hotspots is not None and not hotspots.empty and not combined.empty:
        hz = hotspots[["country", "store_name", "product_id", "product_name", 
                      "claim_probability_pct", "risk_rank"]].copy()
        combined = combined.merge(hz, on=keys, how="left")
    
    # Sort results
    if not combined.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in combined.columns:
            sort_cols.append("risk_rank")
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False] + [True] * (len(sort_cols) - 1)
        combined = combined.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if hotspots is not None and not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
        
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
        
        if not combined.empty:
            combined.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(combined)} rows)")
        else:
            print("Warning: No final forecast data to save")
            
    except Exception as e:
        print(f"Error saving files: {str(e)}")
    
    return combined


# ---------------------------
# Main
# ---------------------------
def run(paths: Dict[str, str], outdir: str = ".", params: Optional[Dict] = None):
    """Main execution function"""
    if params is None:
        params = DEFAULT_PARAMS
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_data(paths)
        
        # Generate hotspots
        print("Generating hotspot analysis...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, 
                                 lookback_days=params["LOOKBACK_DAYS_HOTSPOT"])
        
        # Generate forecasts
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # Blend and save
        print("Blending forecasts and saving results...")
        combined = blend_and_save(hotspots, mvp, pro, params, outdir=outdir)
        
        print("Forecast generation completed successfully!")
        return hotspots, mvp, pro, combined
        
    except Exception as e:
        print(f"Error in forecast generation: {str(e)}")
        raise


if __name__ == "__main__":
    PATHS = {
        "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
        "stores":   "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
        "sales":    "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
        "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
    }
    
    # Optional: set output directory
    OUTDIR = "."
    
    # Run with error handling
    try:
        hotspots, mvp, pro, combined = run(PATHS, outdir=OUTDIR, params=DEFAULT_PARAMS)
        print(f"\nFiles saved in: {OUTDIR}")
        
        if not combined.empty:
            print(f"\nTop 5 forecast recommendations:")
            print(combined[["country", "store_name", "product_name", "final_units_to_prestore", 
                           "horizon_end", "latest_stock_date"]].head().to_string(index=False))
        
    except Exception as e:
        print(f"Execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
Generating hotspot analysis...
Generating MVP forecast...
Generating PRO forecast...
Blending forecasts and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)
Forecast generation completed successfully!

Files saved in: .

Top 5 forecast recommendations:
country      store_name         product_name  final_units_to_prestore horizon_end latest_stock_date
 Turkey    Apple Ankara       iPad (7th Gen)                       28  2024-09-20        2024-09-06
 Turkey  Apple Istanbul       iPad (7th Gen)                       26  2024-09-20        2024-09-06
  India Apple New Delhi Apple Watch Series 5                       24  2024-09-20        2024-09-06
  India Apple New Delhi        iPhone 11 Pro                       24  2024-09-20        2024-09-06
  India Apple New Delhi   

In [2]:
# qc_forecast_playbook.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb (replacement) units
# to pre-stock for the next H days, using MVP (moving average) and PRO
# (lifecycle bucket hazard) approaches.
#
# Inputs (CSV with headers; date columns can be YYYY-MM-DD):
#   products.csv: product_id, product_name, category_id, launch_date, price (optional)
#   stores.csv:   store_id, store_name, city, country
#   sales.csv:    sale_id, sale_date, store_id, product_id, quantity
#   warranty.csv: claim_id, claim_date, sale_id, repair_status (optional)
#
# Outputs (CSV):
#   qc_hotspots.csv            - recent 180d risk ranking by claim probability (store×product)
#   qc_mvp_forecast.csv        - MVP forecast (moving average + safety stock)
#   qc_pro_forecast.csv        - PRO forecast (bucket hazard × installed base)
#   qc_final_forecast.csv      - blended final recommendation with dates
#
# Usage:
#   1) Edit the PATHS dict in __main__ to point to your CSVs.
#   2) Adjust parameters in PARAMS as needed.
#   3) Run:  python qc_forecast_playbook.py
#
# Requires: pandas, numpy (no external ML libs).
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict, Tuple, Optional

# ---------------------------
# Parameters
# ---------------------------
DEFAULT_PARAMS = {
    "HORIZON_DAYS": 30,          # planning horizon window
    "K_MONTHS_MA": 3,            # moving average window (months)
    "LEAD_TIME_DAYS": 14,        # replenishment lead time
    "SERVICE_Z": 1.65,           # 95% service level (z=1.65), 99% -> 2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# hotspot ranking lookback
    "BUCKET_DAYS": [0, 180, 360],# lifecycle buckets: [0-180], [181-360], 361+
    "BLEND_WEIGHT_PRO": 0.7,     # final blend: 70% PRO + 30% MVP (tune as needed)
    "MIN_HISTORY_CLAIMS": 30     # if product total claims < this, treat as sparse history
}


# ---------------------------
# Utilities
# ---------------------------
def _to_date(s):
    """Convert series to date, handling errors gracefully"""
    return pd.to_datetime(s, errors="coerce").dt.date

def _days_between(a, b):
    """Calculate days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def _safe_div(a, b):
    """Safe division avoiding division by zero"""
    return np.where(b == 0, 0.0, a / b)

def _ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

def _validate_file_exists(filepath: str) -> None:
    """Validate that file exists"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

def _validate_required_columns(df: pd.DataFrame, required_cols: list, filename: str) -> None:
    """Validate that required columns exist in dataframe"""
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in {filename}: {missing}")


# ---------------------------
# Loading / Cleaning
# ---------------------------
def load_data(paths: Dict[str, str]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, datetime.date]:
    """Load and validate data from CSV files"""
    
    # Validate files exist
    for name, path in paths.items():
        _validate_file_exists(path)
    
    try:
        # Load data
        products = pd.read_csv(paths["products"])
        stores = pd.read_csv(paths["stores"])
        sales = pd.read_csv(paths["sales"])
        warranty = pd.read_csv(paths["warranty"])
        
        # Validate required columns
        _validate_required_columns(products, ["product_id", "product_name", "category_id"], "products.csv")
        _validate_required_columns(stores, ["store_id", "store_name", "city", "country"], "stores.csv")
        _validate_required_columns(sales, ["sale_id", "sale_date", "store_id", "product_id", "quantity"], "sales.csv")
        _validate_required_columns(warranty, ["claim_id", "claim_date", "sale_id"], "warranty.csv")
        
        # Convert dates
        if "launch_date" in products.columns:
            products["launch_date"] = _to_date(products["launch_date"])
        sales["sale_date"] = _to_date(sales["sale_date"])
        warranty["claim_date"] = _to_date(warranty["claim_date"])
        
        # Basic cleaning
        sales = sales[sales["quantity"] > 0].copy()
        
        # Remove rows with invalid dates
        sales = sales.dropna(subset=["sale_date"])
        warranty = warranty.dropna(subset=["claim_date"])
        
        # Deduplicate
        sales = sales.drop_duplicates(subset=["sale_id"])
        warranty = warranty.drop_duplicates(subset=["claim_id"])
        
        # Calculate TODAY from actual data
        today = max(
            sales["sale_date"].max() if not sales.empty else datetime.now().date(),
            warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
        )
        
        print(f"Data loaded successfully. TODAY calculated as: {today}")
        print(f"Sales records: {len(sales)}, Warranty claims: {len(warranty)}")
        
        return products, stores, sales, warranty, today
        
    except Exception as e:
        raise ValueError(f"Error loading data: {str(e)}")


# ---------------------------
# Joins
# ---------------------------
def join_sales_warranty(products, stores, sales, warranty):
    """Join warranty claims with sales and product/store info"""
    ws = (warranty
          .merge(sales[["sale_id", "sale_date", "store_id", "product_id", "quantity"]], 
                 on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                 on="product_id", how="left")
          .merge(stores[["store_id", "store_name", "city", "country"]], 
                 on="store_id", how="left")
         )
    return ws


# ---------------------------
# Hotspot ranking (recent 180d)
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, lookback_days=180):
    """Build hotspot ranking based on recent claim probability"""
    cutoff = today - timedelta(days=lookback_days)
    
    # Recent sales and warranty claims
    s_recent = sales[sales["sale_date"] >= cutoff].copy()
    w_recent = warranty[warranty["claim_date"] >= cutoff].copy()
    
    if s_recent.empty:
        print("Warning: No recent sales data found for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales
    base = (s_recent
            .merge(products[["product_id", "product_name"]], on="product_id", how="left")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Aggregate claims
    claims = pd.DataFrame()
    if not w_recent.empty:
        claims = (w_recent
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .merge(products[["product_id", "product_name"]], on="product_id", how="left")
                  .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
                  .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
                  .agg(claims=("claim_id", "count"))
                  .reset_index())
    
    # Merge and calculate probabilities
    if claims.empty:
        base["claims"] = 0
        df = base
    else:
        df = base.merge(claims, on=["country", "store_name", "product_id", "product_name"], how="left")
        df["claims"] = df["claims"].fillna(0).astype(int)
    
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    df = df.sort_values(["risk_rank", "country", "store_name", "product_name"])
    
    return df


# ---------------------------
# Installed base & monthly claims
# ---------------------------
def build_installed_base(stores, products, sales, warranty):
    """Calculate installed base (sold - claimed) per store×product"""
    # Cumulative sales
    sold = (sales
            .groupby(["store_id", "product_id"], as_index=False)
            .agg(units_sold=("quantity", "sum")))
    
    # Cumulative claims
    claims = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    if not warranty.empty:
        claims = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .groupby(["store_id", "product_id"], as_index=False)
                  .agg(units_claimed=("claim_id", "count")))
    
    # Merge and calculate installed base
    base = sold.merge(claims, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0).astype(int)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                   on="product_id", how="left"))
    
    return base[["country", "store_name", "product_id", "product_name", 
                 "category_id", "launch_date", "installed_base"]]


def build_claims_monthly(products, stores, sales, warranty):
    """Build monthly claims aggregation"""
    if warranty.empty:
        return pd.DataFrame()
    
    ws = join_sales_warranty(products, stores, sales, warranty)
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    grp = (ws.groupby(["country", "store_name", "product_id", "product_name", "month"], dropna=False)
             .agg(claims=("claim_id", "count")).reset_index())
    
    return grp


# ---------------------------
# MVP forecast (moving average + safety)
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Generate MVP forecast using moving average approach"""
    horizon_days = params["HORIZON_DAYS"]
    k_months = params["K_MONTHS_MA"]
    z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    
    cm = build_claims_monthly(products, stores, sales, warranty)
    
    if cm.empty:
        print("Warning: No claims data available for MVP forecast")
        return pd.DataFrame()
    
    # Keep last k months for stats
    cutoff_period = pd.Timestamp(today).to_period("M") - k_months
    cm["month"] = pd.to_datetime(cm["month"].astype(str))
    cm["month_period"] = cm["month"].dt.to_period("M")
    recent = cm[cm["month_period"] >= cutoff_period]
    
    if recent.empty:
        print("Warning: No recent claims data for MVP forecast")
        return pd.DataFrame()
    
    # Calculate statistics
    stats = (recent
             .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
             .agg(ma_k="mean", sd_k="std").reset_index())
    stats["sd_k"] = stats["sd_k"].fillna(0.0)
    
    # Convert monthly to horizon days
    stats["demand_horizon"] = _ensure_nonneg_int(np.ceil(stats["ma_k"] * (horizon_days / 30.0)))
    stats["safety_stock"] = _ensure_nonneg_int(np.ceil(z * stats["sd_k"] * np.sqrt(lead_time / 30.0)))
    stats["units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    stats["horizon_end"] = horizon_end
    stats["latest_stock_date"] = latest_stock_date
    
    # Rename columns and sort
    out = stats.rename(columns={
        "ma_k": "mvp_ma_month",
        "sd_k": "mvp_sd_month",
        "demand_horizon": "mvp_demand_horizon",
        "safety_stock": "mvp_safety",
        "units_to_prestore": "mvp_units_to_prestore"
    })
    
    out = out.sort_values(["mvp_units_to_prestore", "country", "store_name", "product_name"], 
                         ascending=[False, True, True, True])
    
    return out


# ---------------------------
# PRO forecast (bucket hazard × installed base)
# ---------------------------
def build_product_claim_prob(products, stores, sales, warranty):
    """Calculate product-level claim probability"""
    # Total sales per product
    sold = (sales.groupby(["product_id"], as_index=False)
                 .agg(units_sold=("quantity", "sum")))
    
    # Total claims per product
    claims = pd.DataFrame(columns=["product_id", "claims"])
    if not warranty.empty:
        claims = (warranty.merge(sales[["sale_id", "product_id"]], on="sale_id", how="left")
                         .groupby(["product_id"], as_index=False)
                         .agg(claims=("claim_id", "count")))
    
    # Merge and calculate probability
    df = sold.merge(claims, on="product_id", how="left")
    df["claims"] = df["claims"].fillna(0).astype(int)
    df["claim_prob"] = _safe_div(df["claims"], df["units_sold"])
    
    # Add product info
    df = df.merge(products[["product_id", "product_name", "category_id"]], on="product_id", how="left")
    
    return df[["product_id", "product_name", "category_id", "claim_prob", "claims", "units_sold"]]


def build_lifecycle_bucket_claims(products, sales, warranty, params):
    """Count claims by lifecycle bucket per product"""
    if warranty.empty:
        return pd.DataFrame()
    
    buckets = params["BUCKET_DAYS"]
    ws = (warranty
          .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "launch_date"]], on="product_id", how="left"))
    
    ws["age_days"] = _days_between(ws["claim_date"], ws["launch_date"])
    
    def bucket_label(d):
        if pd.isna(d):
            return "unknown"
        d = int(d)
        if d < 0:
            return "prelaunch"
        if d <= buckets[1]:    # 0-180
            return "0-6"
        if d <= buckets[2]:    # 181-360
            return "7-12"
        return "13+"
    
    ws["bucket"] = ws["age_days"].apply(bucket_label)
    c = (ws.groupby(["product_id", "bucket"], dropna=False)
           .agg(claims=("claim_id", "count")).reset_index())
    
    return c


def pro_forecast(products, stores, sales, warranty, today, params):
    """Generate PRO forecast using lifecycle bucket approach"""
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"]
    z = params["SERVICE_Z"]
    buckets = params["BUCKET_DAYS"]
    min_hist = params["MIN_HISTORY_CLAIMS"]
    k_months = params["K_MONTHS_MA"]
    
    # Get installed base
    base = build_installed_base(stores, products, sales, warranty)
    
    if base.empty:
        print("Warning: No installed base data for PRO forecast")
        return pd.DataFrame()
    
    # Product-level probabilities
    prod_prob = build_product_claim_prob(products, stores, sales, warranty)
    bucket_claims = build_lifecycle_bucket_claims(products, sales, warranty, params)
    
    # Handle empty bucket claims
    if bucket_claims.empty:
        # Create dummy bucket data
        unique_products = prod_prob["product_id"].unique()
        bucket_data = []
        for pid in unique_products:
            for bucket in ["0-6", "7-12", "13+"]:
                bucket_data.append({"product_id": pid, "bucket": bucket, "claims": 0})
        bucket_claims = pd.DataFrame(bucket_data)
    
    # Pivot bucket claims
    pivot = bucket_claims.pivot_table(index="product_id", columns="bucket", values="claims", 
                                     aggfunc="sum", fill_value=0).reset_index()
    
    # Merge with product probabilities
    prod = prod_prob.merge(pivot, on="product_id", how="left").fillna(0)
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in prod.columns:
            prod[b] = 0
    
    prod["total_claims_hist"] = prod[["0-6", "7-12", "13+"]].sum(axis=1)
    
    # Calculate monthly rates per bucket
    def compute_monthly_rates(row):
        total = row["total_claims_hist"]
        cp = row["claim_prob"]
        
        # Calculate shares
        if total > 0:
            s0 = row["0-6"] / total
            s1 = row["7-12"] / total
            s2 = row["13+"] / total
        else:
            s0 = s1 = s2 = 1/3.0
        
        # Monthly rates
        r0 = cp * s0 / 6.0    # 0-6 months
        r1 = cp * s1 / 6.0    # 7-12 months
        r2 = cp * s2 / 999.0  # 13+ months (very low rate)
        
        return pd.Series({"rate_0_6": r0, "rate_7_12": r1, "rate_13p": r2})
    
    rates = prod.apply(compute_monthly_rates, axis=1)
    prod = pd.concat([prod, rates], axis=1)
    
    # Determine current age bucket for each store×product
    base["age_days_now"] = _days_between(today, base["launch_date"])
    
    def current_bucket(d):
        if pd.isna(d):
            return "13+"
        d = int(d)
        if d <= buckets[1]:
            return "0-6"
        if d <= buckets[2]:
            return "7-12"
        return "13+"
    
    base["current_bucket"] = base["age_days_now"].apply(current_bucket)
    
    # Aggregate exposure by bucket
    exposure = (base
                .groupby(["country", "store_name", "product_id", "product_name", "current_bucket"], 
                        dropna=False)
                .agg(units=("installed_base", "sum"))
                .reset_index())
    
    # Pivot exposure
    expo_pivot = (exposure
                  .pivot_table(index=["country", "store_name", "product_id", "product_name"],
                              columns="current_bucket", values="units", aggfunc="sum", fill_value=0)
                  .reset_index())
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in expo_pivot.columns:
            expo_pivot[b] = 0
    
    # Merge with rates
    expo_pivot = expo_pivot.merge(prod[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims_hist"]], 
                                 on="product_id", how="left")
    
    # Handle sparse history
    sparse_mask = expo_pivot["total_claims_hist"].fillna(0) < min_hist
    for col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        expo_pivot.loc[sparse_mask, col] = expo_pivot.loc[sparse_mask, col] * 1.25
    
    # Calculate expected claims
    factor = horizon_days / 30.0
    expo_pivot["expected_claims"] = (
        expo_pivot["0-6"] * (expo_pivot["rate_0_6"] * factor) +
        expo_pivot["7-12"] * (expo_pivot["rate_7_12"] * factor) +
        expo_pivot["13+"] * (expo_pivot["rate_13p"] * factor)
    )
    
    # Safety stock calculation
    cm = build_claims_monthly(products, stores, sales, warranty)
    if not cm.empty:
        cutoff_period = pd.Timestamp(today).to_period("M") - k_months
        cm["month"] = pd.to_datetime(cm["month"].astype(str))
        cm["month_period"] = cm["month"].dt.to_period("M")
        recent = cm[cm["month_period"] >= cutoff_period]
        
        sd = (recent.groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
                    .std().reset_index().rename(columns={"claims": "sd_month"}))
        sd["sd_month"] = sd["sd_month"].fillna(0.0)
    else:
        sd = pd.DataFrame(columns=["country", "store_name", "product_id", "product_name", "sd_month"])
    
    # Merge safety stock data
    out = expo_pivot.merge(sd, on=["country", "store_name", "product_id", "product_name"], how="left")
    out["sd_month"] = out["sd_month"].fillna(0.0)
    
    # Calculate final values
    out["safety_stock"] = _ensure_nonneg_int(np.ceil(z * out["sd_month"] * np.sqrt(lead_time / 30.0)))
    out["expected_claims"] = _ensure_nonneg_int(np.ceil(out["expected_claims"]))
    out["pro_units_to_prestore"] = out["expected_claims"] + out["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    out["horizon_end"] = horizon_end
    out["latest_stock_date"] = latest_stock_date
    
    return out[["country", "store_name", "product_id", "product_name",
                "expected_claims", "safety_stock", "pro_units_to_prestore",
                "horizon_end", "latest_stock_date"]]


# ---------------------------
# Final blend & save
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save results"""
    w = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty dataframes
    if mvp.empty and pro.empty:
        print("Warning: Both MVP and PRO forecasts are empty")
        combined = pd.DataFrame()
    elif mvp.empty:
        combined = pro.copy()
        combined["final_units_to_prestore"] = combined["pro_units_to_prestore"]
    elif pro.empty:
        combined = mvp.copy()
        combined["final_units_to_prestore"] = combined["mvp_units_to_prestore"]
    else:
        # Merge forecasts
        combined = pd.merge(pro, mvp, on=keys, how="outer")
        
        # Fill NaNs
        for col in ["pro_units_to_prestore", "mvp_units_to_prestore", "expected_claims", 
                   "mvp_demand_horizon", "mvp_safety"]:
            if col in combined.columns:
                combined[col] = combined[col].fillna(0)
        
        # Blend forecasts
        combined["final_units_to_prestore"] = _ensure_nonneg_int(
            np.ceil(w * combined["pro_units_to_prestore"] + (1-w) * combined["mvp_units_to_prestore"])
        )
        
        # Handle date columns
        if "horizon_end_x" in combined.columns and "horizon_end_y" in combined.columns:
            combined["horizon_end"] = combined["horizon_end_x"].fillna(combined["horizon_end_y"])
            combined["latest_stock_date"] = combined["latest_stock_date_x"].fillna(combined["latest_stock_date_y"])
            drop_cols = [c for c in combined.columns if c.endswith("_x") or c.endswith("_y")]
            combined = combined.drop(columns=drop_cols)
    
    # Add hotspot data if available
    if hotspots is not None and not hotspots.empty and not combined.empty:
        hz = hotspots[["country", "store_name", "product_id", "product_name", 
                      "claim_probability_pct", "risk_rank"]].copy()
        combined = combined.merge(hz, on=keys, how="left")
    
    # Sort results
    if not combined.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in combined.columns:
            sort_cols.append("risk_rank")
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False] + [True] * (len(sort_cols) - 1)
        combined = combined.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if hotspots is not None and not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
        
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
        
        if not combined.empty:
            combined.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(combined)} rows)")
        else:
            print("Warning: No final forecast data to save")
            
    except Exception as e:
        print(f"Error saving files: {str(e)}")
    
    return combined


# ---------------------------
# Main
# ---------------------------
def run(paths: Dict[str, str], outdir: str = ".", params: Optional[Dict] = None):
    """Main execution function"""
    if params is None:
        params = DEFAULT_PARAMS
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_data(paths)
        
        # Generate hotspots
        print("Generating hotspot analysis...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, 
                                 lookback_days=params["LOOKBACK_DAYS_HOTSPOT"])
        
        # Generate forecasts
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # Blend and save
        print("Blending forecasts and saving results...")
        combined = blend_and_save(hotspots, mvp, pro, params, outdir=outdir)
        
        print("Forecast generation completed successfully!")
        return hotspots, mvp, pro, combined
        
    except Exception as e:
        print(f"Error in forecast generation: {str(e)}")
        raise


if __name__ == "__main__":
    PATHS = {
        "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
        "stores":   "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
        "sales":    "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
        "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
    }
    
    # Optional: set output directory
    OUTDIR = "."
    
    # Run with error handling
    try:
        hotspots, mvp, pro, combined, iphone_strategy = run(PATHS, outdir=OUTDIR, params=DEFAULT_PARAMS)
        print(f"\nFiles saved in: {OUTDIR}")
        
        if not combined.empty:
            print(f"\n📦 REFURBISH INVENTORY DISTRIBUTION PLAN (Target Date: September 20, 2024)")
            print("="*80)
            
            # Filter for items that need restocking
            restock_needed = combined[combined["final_units_to_prestore"] > 0].copy()
            
            if not restock_needed.empty:
                # Country-level summary
                country_summary = (restock_needed
                                 .groupby("country")
                                 .agg(total_units=("final_units_to_prestore", "sum"),
                                      stores_count=("store_name", "nunique"),
                                      products_count=("product_name", "nunique"))
                                 .reset_index()
                                 .sort_values("total_units", ascending=False))
                
                print("\n🌍 COUNTRY-LEVEL ALLOCATION:")
                print("-" * 50)
                for _, row in country_summary.iterrows():
                    print(f"{row['country']:15} → {row['total_units']:4d} units "
                          f"({row['stores_count']} stores, {row['products_count']} products)")
                
                total_global_units = country_summary["total_units"].sum()
                print(f"\n🎯 TOTAL GLOBAL ALLOCATION: {total_global_units:,} refurb units")
                
                # High-priority stores (>50 units needed)
                high_priority = (restock_needed
                               .groupby(["country", "store_name"])
                               .agg(store_total=("final_units_to_prestore", "sum"))
                               .reset_index()
                               .query("store_total >= 50")
                               .sort_values("store_total", ascending=False))
                
                if not high_priority.empty:
                    print(f"\n🚨 HIGH-PRIORITY STORES (50+ units needed):")
                    print("-" * 50)
                    for _, row in high_priority.iterrows():
                        print(f"{row['country']} - {row['store_name']:30} → {row['store_total']:3d} units")
                
                # Product category breakdown
                if "category_id" in restock_needed.columns:
                    category_breakdown = (restock_needed
                                        .groupby("category_id")
                                        .agg(units=("final_units_to_prestore", "sum"))
                                        .reset_index()
                                        .sort_values("units", ascending=False))
                    
                    print(f"\n📱 PRODUCT CATEGORY BREAKDOWN:")
                    print("-" * 50)
                    for _, row in category_breakdown.iterrows():
                        pct = (row['units'] / total_global_units) * 100
                        print(f"{row['category_id']:15} → {row['units']:4d} units ({pct:.1f}%)")
                
                # Urgent shipments (need to ship by latest_stock_date)
                from datetime import datetime
                target_date = datetime(2024, 9, 20).date()
                
                urgent_mask = pd.to_datetime(restock_needed["latest_stock_date"]).dt.date <= target_date
                urgent_shipments = restock_needed[urgent_mask].copy()
                
                if not urgent_shipments.empty:
                    urgent_summary = (urgent_shipments
                                    .groupby(["country", "store_name"])
                                    .agg(urgent_units=("final_units_to_prestore", "sum"),
                                         ship_by=("latest_stock_date", "max"))
                                    .reset_index()
                                    .sort_values("urgent_units", ascending=False))
                    
                    print(f"\n⚡ URGENT SHIPMENTS (Ship by Sept 20):")
                    print("-" * 50)
                    for _, row in urgent_summary.head(10).iterrows():
                        print(f"{row['country']} - {row['store_name']:25} → {row['urgent_units']:3d} units "
                              f"(by {row['ship_by']})")
                    
                    if len(urgent_summary) > 10:
                        remaining_urgent = urgent_summary.iloc[10:]["urgent_units"].sum()
                        print(f"... and {len(urgent_summary)-10} more stores needing {remaining_urgent} units")
                
                print(f"\n📊 DISTRIBUTION LOGISTICS SUMMARY:")
                print("-" * 50)
                print(f"• Total shipments needed: {len(restock_needed):,} store×product combinations")
                print(f"• Countries to serve: {restock_needed['country'].nunique()}")
                print(f"• Stores requiring stock: {restock_needed['store_name'].nunique()}")
                print(f"• Unique products: {restock_needed['product_name'].nunique()}")
                
            else:
                print("✅ No refurbish inventory needed - all stores sufficiently stocked!")
        
    except Exception as e:
        print(f"Execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
Generating hotspot analysis...
Generating MVP forecast...
Generating PRO forecast...
Blending forecasts and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)
Forecast generation completed successfully!
Execution failed: not enough values to unpack (expected 5, got 4)


Traceback (most recent call last):
  File "/var/folders/zt/vccr42bs5db2sjm4g8nyfcyr0000gn/T/ipykernel_94038/4119321897.py", line 645, in <module>
    hotspots, mvp, pro, combined, iphone_strategy = run(PATHS, outdir=OUTDIR, params=DEFAULT_PARAMS)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: not enough values to unpack (expected 5, got 4)


In [3]:
# qc_forecast_playbook.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb (replacement) units
# to pre-stock for the next H days, using MVP (moving average) and PRO
# (lifecycle bucket hazard) approaches.
#
# Inputs (CSV with headers; date columns can be YYYY-MM-DD):
#   products.csv: product_id, product_name, category_id, launch_date, price (optional)
#   stores.csv:   store_id, store_name, city, country
#   sales.csv:    sale_id, sale_date, store_id, product_id, quantity
#   warranty.csv: claim_id, claim_date, sale_id, repair_status (optional)
#
# Outputs (CSV):
#   qc_hotspots.csv            - recent 180d risk ranking by claim probability (store×product)
#   qc_mvp_forecast.csv        - MVP forecast (moving average + safety stock)
#   qc_pro_forecast.csv        - PRO forecast (bucket hazard × installed base)
#   qc_final_forecast.csv      - blended final recommendation with dates
#
# Usage:
#   1) Edit the PATHS dict in __main__ to point to your CSVs.
#   2) Adjust parameters in PARAMS as needed.
#   3) Run:  python qc_forecast_playbook.py
#
# Requires: pandas, numpy (no external ML libs).
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict, Tuple, Optional

# ---------------------------
# Parameters
# ---------------------------
DEFAULT_PARAMS = {
    "HORIZON_DAYS": 30,          # planning horizon window
    "K_MONTHS_MA": 3,            # moving average window (months)
    "LEAD_TIME_DAYS": 14,        # replenishment lead time
    "SERVICE_Z": 1.65,           # 95% service level (z=1.65), 99% -> 2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# hotspot ranking lookback
    "BUCKET_DAYS": [0, 180, 360],# lifecycle buckets: [0-180], [181-360], 361+
    "BLEND_WEIGHT_PRO": 0.7,     # final blend: 70% PRO + 30% MVP (tune as needed)
    "MIN_HISTORY_CLAIMS": 30     # if product total claims < this, treat as sparse history
}


# ---------------------------
# Utilities
# ---------------------------
def _to_date(s):
    """Convert series to date, handling errors gracefully"""
    return pd.to_datetime(s, errors="coerce").dt.date

def _days_between(a, b):
    """Calculate days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def _safe_div(a, b):
    """Safe division avoiding division by zero"""
    return np.where(b == 0, 0.0, a / b)

def _ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

def _validate_file_exists(filepath: str) -> None:
    """Validate that file exists"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

def _validate_required_columns(df: pd.DataFrame, required_cols: list, filename: str) -> None:
    """Validate that required columns exist in dataframe"""
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in {filename}: {missing}")


# ---------------------------
# Loading / Cleaning
# ---------------------------
def load_data(paths: Dict[str, str]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, datetime.date]:
    """Load and validate data from CSV files"""
    
    # Validate files exist
    for name, path in paths.items():
        _validate_file_exists(path)
    
    try:
        # Load data
        products = pd.read_csv(paths["products"])
        stores = pd.read_csv(paths["stores"])
        sales = pd.read_csv(paths["sales"])
        warranty = pd.read_csv(paths["warranty"])
        
        # Validate required columns
        _validate_required_columns(products, ["product_id", "product_name", "category_id"], "products.csv")
        _validate_required_columns(stores, ["store_id", "store_name", "city", "country"], "stores.csv")
        _validate_required_columns(sales, ["sale_id", "sale_date", "store_id", "product_id", "quantity"], "sales.csv")
        _validate_required_columns(warranty, ["claim_id", "claim_date", "sale_id"], "warranty.csv")
        
        # Convert dates
        if "launch_date" in products.columns:
            products["launch_date"] = _to_date(products["launch_date"])
        sales["sale_date"] = _to_date(sales["sale_date"])
        warranty["claim_date"] = _to_date(warranty["claim_date"])
        
        # Basic cleaning
        sales = sales[sales["quantity"] > 0].copy()
        
        # Remove rows with invalid dates
        sales = sales.dropna(subset=["sale_date"])
        warranty = warranty.dropna(subset=["claim_date"])
        
        # Deduplicate
        sales = sales.drop_duplicates(subset=["sale_id"])
        warranty = warranty.drop_duplicates(subset=["claim_id"])
        
        # Calculate TODAY from actual data
        today = max(
            sales["sale_date"].max() if not sales.empty else datetime.now().date(),
            warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
        )
        
        print(f"Data loaded successfully. TODAY calculated as: {today}")
        print(f"Sales records: {len(sales)}, Warranty claims: {len(warranty)}")
        
        return products, stores, sales, warranty, today
        
    except Exception as e:
        raise ValueError(f"Error loading data: {str(e)}")


# ---------------------------
# Joins
# ---------------------------
def join_sales_warranty(products, stores, sales, warranty):
    """Join warranty claims with sales and product/store info"""
    ws = (warranty
          .merge(sales[["sale_id", "sale_date", "store_id", "product_id", "quantity"]], 
                 on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                 on="product_id", how="left")
          .merge(stores[["store_id", "store_name", "city", "country"]], 
                 on="store_id", how="left")
         )
    return ws


# ---------------------------
# Hotspot ranking (recent 180d)
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, lookback_days=180):
    """Build hotspot ranking based on recent claim probability"""
    cutoff = today - timedelta(days=lookback_days)
    
    # Recent sales and warranty claims
    s_recent = sales[sales["sale_date"] >= cutoff].copy()
    w_recent = warranty[warranty["claim_date"] >= cutoff].copy()
    
    if s_recent.empty:
        print("Warning: No recent sales data found for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales
    base = (s_recent
            .merge(products[["product_id", "product_name"]], on="product_id", how="left")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Aggregate claims
    claims = pd.DataFrame()
    if not w_recent.empty:
        claims = (w_recent
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .merge(products[["product_id", "product_name"]], on="product_id", how="left")
                  .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
                  .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
                  .agg(claims=("claim_id", "count"))
                  .reset_index())
    
    # Merge and calculate probabilities
    if claims.empty:
        base["claims"] = 0
        df = base
    else:
        df = base.merge(claims, on=["country", "store_name", "product_id", "product_name"], how="left")
        df["claims"] = df["claims"].fillna(0).astype(int)
    
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    df = df.sort_values(["risk_rank", "country", "store_name", "product_name"])
    
    return df


# ---------------------------
# Installed base & monthly claims
# ---------------------------
def build_installed_base(stores, products, sales, warranty):
    """Calculate installed base (sold - claimed) per store×product"""
    # Cumulative sales
    sold = (sales
            .groupby(["store_id", "product_id"], as_index=False)
            .agg(units_sold=("quantity", "sum")))
    
    # Cumulative claims
    claims = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    if not warranty.empty:
        claims = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .groupby(["store_id", "product_id"], as_index=False)
                  .agg(units_claimed=("claim_id", "count")))
    
    # Merge and calculate installed base
    base = sold.merge(claims, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0).astype(int)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                   on="product_id", how="left"))
    
    return base[["country", "store_name", "product_id", "product_name", 
                 "category_id", "launch_date", "installed_base"]]


def build_claims_monthly(products, stores, sales, warranty):
    """Build monthly claims aggregation"""
    if warranty.empty:
        return pd.DataFrame()
    
    ws = join_sales_warranty(products, stores, sales, warranty)
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    grp = (ws.groupby(["country", "store_name", "product_id", "product_name", "month"], dropna=False)
             .agg(claims=("claim_id", "count")).reset_index())
    
    return grp


# ---------------------------
# MVP forecast (moving average + safety)
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Generate MVP forecast using moving average approach"""
    horizon_days = params["HORIZON_DAYS"]
    k_months = params["K_MONTHS_MA"]
    z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    
    cm = build_claims_monthly(products, stores, sales, warranty)
    
    if cm.empty:
        print("Warning: No claims data available for MVP forecast")
        return pd.DataFrame()
    
    # Keep last k months for stats
    cutoff_period = pd.Timestamp(today).to_period("M") - k_months
    cm["month"] = pd.to_datetime(cm["month"].astype(str))
    cm["month_period"] = cm["month"].dt.to_period("M")
    recent = cm[cm["month_period"] >= cutoff_period]
    
    if recent.empty:
        print("Warning: No recent claims data for MVP forecast")
        return pd.DataFrame()
    
    # Calculate statistics
    stats = (recent
             .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
             .agg(ma_k="mean", sd_k="std").reset_index())
    stats["sd_k"] = stats["sd_k"].fillna(0.0)
    
    # Convert monthly to horizon days
    stats["demand_horizon"] = _ensure_nonneg_int(np.ceil(stats["ma_k"] * (horizon_days / 30.0)))
    stats["safety_stock"] = _ensure_nonneg_int(np.ceil(z * stats["sd_k"] * np.sqrt(lead_time / 30.0)))
    stats["units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    stats["horizon_end"] = horizon_end
    stats["latest_stock_date"] = latest_stock_date
    
    # Rename columns and sort
    out = stats.rename(columns={
        "ma_k": "mvp_ma_month",
        "sd_k": "mvp_sd_month",
        "demand_horizon": "mvp_demand_horizon",
        "safety_stock": "mvp_safety",
        "units_to_prestore": "mvp_units_to_prestore"
    })
    
    out = out.sort_values(["mvp_units_to_prestore", "country", "store_name", "product_name"], 
                         ascending=[False, True, True, True])
    
    return out


# ---------------------------
# PRO forecast (bucket hazard × installed base)
# ---------------------------
def build_product_claim_prob(products, stores, sales, warranty):
    """Calculate product-level claim probability"""
    # Total sales per product
    sold = (sales.groupby(["product_id"], as_index=False)
                 .agg(units_sold=("quantity", "sum")))
    
    # Total claims per product
    claims = pd.DataFrame(columns=["product_id", "claims"])
    if not warranty.empty:
        claims = (warranty.merge(sales[["sale_id", "product_id"]], on="sale_id", how="left")
                         .groupby(["product_id"], as_index=False)
                         .agg(claims=("claim_id", "count")))
    
    # Merge and calculate probability
    df = sold.merge(claims, on="product_id", how="left")
    df["claims"] = df["claims"].fillna(0).astype(int)
    df["claim_prob"] = _safe_div(df["claims"], df["units_sold"])
    
    # Add product info
    df = df.merge(products[["product_id", "product_name", "category_id"]], on="product_id", how="left")
    
    return df[["product_id", "product_name", "category_id", "claim_prob", "claims", "units_sold"]]


def build_lifecycle_bucket_claims(products, sales, warranty, params):
    """Count claims by lifecycle bucket per product"""
    if warranty.empty:
        return pd.DataFrame()
    
    buckets = params["BUCKET_DAYS"]
    ws = (warranty
          .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "launch_date"]], on="product_id", how="left"))
    
    ws["age_days"] = _days_between(ws["claim_date"], ws["launch_date"])
    
    def bucket_label(d):
        if pd.isna(d):
            return "unknown"
        d = int(d)
        if d < 0:
            return "prelaunch"
        if d <= buckets[1]:    # 0-180
            return "0-6"
        if d <= buckets[2]:    # 181-360
            return "7-12"
        return "13+"
    
    ws["bucket"] = ws["age_days"].apply(bucket_label)
    c = (ws.groupby(["product_id", "bucket"], dropna=False)
           .agg(claims=("claim_id", "count")).reset_index())
    
    return c


def pro_forecast(products, stores, sales, warranty, today, params):
    """Generate PRO forecast using lifecycle bucket approach"""
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"]
    z = params["SERVICE_Z"]
    buckets = params["BUCKET_DAYS"]
    min_hist = params["MIN_HISTORY_CLAIMS"]
    k_months = params["K_MONTHS_MA"]
    
    # Get installed base
    base = build_installed_base(stores, products, sales, warranty)
    
    if base.empty:
        print("Warning: No installed base data for PRO forecast")
        return pd.DataFrame()
    
    # Product-level probabilities
    prod_prob = build_product_claim_prob(products, stores, sales, warranty)
    bucket_claims = build_lifecycle_bucket_claims(products, sales, warranty, params)
    
    # Handle empty bucket claims
    if bucket_claims.empty:
        # Create dummy bucket data
        unique_products = prod_prob["product_id"].unique()
        bucket_data = []
        for pid in unique_products:
            for bucket in ["0-6", "7-12", "13+"]:
                bucket_data.append({"product_id": pid, "bucket": bucket, "claims": 0})
        bucket_claims = pd.DataFrame(bucket_data)
    
    # Pivot bucket claims
    pivot = bucket_claims.pivot_table(index="product_id", columns="bucket", values="claims", 
                                     aggfunc="sum", fill_value=0).reset_index()
    
    # Merge with product probabilities
    prod = prod_prob.merge(pivot, on="product_id", how="left").fillna(0)
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in prod.columns:
            prod[b] = 0
    
    prod["total_claims_hist"] = prod[["0-6", "7-12", "13+"]].sum(axis=1)
    
    # Calculate monthly rates per bucket
    def compute_monthly_rates(row):
        total = row["total_claims_hist"]
        cp = row["claim_prob"]
        
        # Calculate shares
        if total > 0:
            s0 = row["0-6"] / total
            s1 = row["7-12"] / total
            s2 = row["13+"] / total
        else:
            s0 = s1 = s2 = 1/3.0
        
        # Monthly rates
        r0 = cp * s0 / 6.0    # 0-6 months
        r1 = cp * s1 / 6.0    # 7-12 months
        r2 = cp * s2 / 999.0  # 13+ months (very low rate)
        
        return pd.Series({"rate_0_6": r0, "rate_7_12": r1, "rate_13p": r2})
    
    rates = prod.apply(compute_monthly_rates, axis=1)
    prod = pd.concat([prod, rates], axis=1)
    
    # Determine current age bucket for each store×product
    base["age_days_now"] = _days_between(today, base["launch_date"])
    
    def current_bucket(d):
        if pd.isna(d):
            return "13+"
        d = int(d)
        if d <= buckets[1]:
            return "0-6"
        if d <= buckets[2]:
            return "7-12"
        return "13+"
    
    base["current_bucket"] = base["age_days_now"].apply(current_bucket)
    
    # Aggregate exposure by bucket
    exposure = (base
                .groupby(["country", "store_name", "product_id", "product_name", "current_bucket"], 
                        dropna=False)
                .agg(units=("installed_base", "sum"))
                .reset_index())
    
    # Pivot exposure
    expo_pivot = (exposure
                  .pivot_table(index=["country", "store_name", "product_id", "product_name"],
                              columns="current_bucket", values="units", aggfunc="sum", fill_value=0)
                  .reset_index())
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in expo_pivot.columns:
            expo_pivot[b] = 0
    
    # Merge with rates
    expo_pivot = expo_pivot.merge(prod[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims_hist"]], 
                                 on="product_id", how="left")
    
    # Handle sparse history
    sparse_mask = expo_pivot["total_claims_hist"].fillna(0) < min_hist
    for col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        expo_pivot.loc[sparse_mask, col] = expo_pivot.loc[sparse_mask, col] * 1.25
    
    # Calculate expected claims
    factor = horizon_days / 30.0
    expo_pivot["expected_claims"] = (
        expo_pivot["0-6"] * (expo_pivot["rate_0_6"] * factor) +
        expo_pivot["7-12"] * (expo_pivot["rate_7_12"] * factor) +
        expo_pivot["13+"] * (expo_pivot["rate_13p"] * factor)
    )
    
    # Safety stock calculation
    cm = build_claims_monthly(products, stores, sales, warranty)
    if not cm.empty:
        cutoff_period = pd.Timestamp(today).to_period("M") - k_months
        cm["month"] = pd.to_datetime(cm["month"].astype(str))
        cm["month_period"] = cm["month"].dt.to_period("M")
        recent = cm[cm["month_period"] >= cutoff_period]
        
        sd = (recent.groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
                    .std().reset_index().rename(columns={"claims": "sd_month"}))
        sd["sd_month"] = sd["sd_month"].fillna(0.0)
    else:
        sd = pd.DataFrame(columns=["country", "store_name", "product_id", "product_name", "sd_month"])
    
    # Merge safety stock data
    out = expo_pivot.merge(sd, on=["country", "store_name", "product_id", "product_name"], how="left")
    out["sd_month"] = out["sd_month"].fillna(0.0)
    
    # Calculate final values
    out["safety_stock"] = _ensure_nonneg_int(np.ceil(z * out["sd_month"] * np.sqrt(lead_time / 30.0)))
    out["expected_claims"] = _ensure_nonneg_int(np.ceil(out["expected_claims"]))
    out["pro_units_to_prestore"] = out["expected_claims"] + out["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    out["horizon_end"] = horizon_end
    out["latest_stock_date"] = latest_stock_date
    
    return out[["country", "store_name", "product_id", "product_name",
                "expected_claims", "safety_stock", "pro_units_to_prestore",
                "horizon_end", "latest_stock_date"]]


# ---------------------------
# Final blend & save
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save results"""
    w = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty dataframes
    if mvp.empty and pro.empty:
        print("Warning: Both MVP and PRO forecasts are empty")
        combined = pd.DataFrame()
    elif mvp.empty:
        combined = pro.copy()
        combined["final_units_to_prestore"] = combined["pro_units_to_prestore"]
    elif pro.empty:
        combined = mvp.copy()
        combined["final_units_to_prestore"] = combined["mvp_units_to_prestore"]
    else:
        # Merge forecasts
        combined = pd.merge(pro, mvp, on=keys, how="outer")
        
        # Fill NaNs
        for col in ["pro_units_to_prestore", "mvp_units_to_prestore", "expected_claims", 
                   "mvp_demand_horizon", "mvp_safety"]:
            if col in combined.columns:
                combined[col] = combined[col].fillna(0)
        
        # Blend forecasts
        combined["final_units_to_prestore"] = _ensure_nonneg_int(
            np.ceil(w * combined["pro_units_to_prestore"] + (1-w) * combined["mvp_units_to_prestore"])
        )
        
        # Handle date columns
        if "horizon_end_x" in combined.columns and "horizon_end_y" in combined.columns:
            combined["horizon_end"] = combined["horizon_end_x"].fillna(combined["horizon_end_y"])
            combined["latest_stock_date"] = combined["latest_stock_date_x"].fillna(combined["latest_stock_date_y"])
            drop_cols = [c for c in combined.columns if c.endswith("_x") or c.endswith("_y")]
            combined = combined.drop(columns=drop_cols)
    
    # Add hotspot data if available
    if hotspots is not None and not hotspots.empty and not combined.empty:
        hz = hotspots[["country", "store_name", "product_id", "product_name", 
                      "claim_probability_pct", "risk_rank"]].copy()
        combined = combined.merge(hz, on=keys, how="left")
    
    # Sort results
    if not combined.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in combined.columns:
            sort_cols.append("risk_rank")
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False] + [True] * (len(sort_cols) - 1)
        combined = combined.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if hotspots is not None and not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
        
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
        
        if not combined.empty:
            combined.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(combined)} rows)")
        else:
            print("Warning: No final forecast data to save")
            
    except Exception as e:
        print(f"Error saving files: {str(e)}")
    
    return combined


# ---------------------------
# Main
# ---------------------------
def run(paths: Dict[str, str], outdir: str = ".", params: Optional[Dict] = None):
    """Main execution function"""
    if params is None:
        params = DEFAULT_PARAMS
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_data(paths)
        
        # Generate hotspots
        print("Generating hotspot analysis...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, 
                                 lookback_days=params["LOOKBACK_DAYS_HOTSPOT"])
        
        # Generate forecasts
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # Blend and save
        print("Blending forecasts and saving results...")
        combined = blend_and_save(hotspots, mvp, pro, params, outdir=outdir)
        
        print("Forecast generation completed successfully!")
        return hotspots, mvp, pro, combined
        
    except Exception as e:
        print(f"Error in forecast generation: {str(e)}")
        raise


if __name__ == "__main__":
    PATHS = {
        "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
        "stores":   "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
        "sales":    "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
        "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
    }
    
    # Optional: set output directory
    OUTDIR = "."
    
    # Run with error handling
    try:
        hotspots, mvp, pro, combined = run(PATHS, outdir=OUTDIR, params=DEFAULT_PARAMS)
        
        # Separate iPhone analysis
        print("\n" + "="*60)
        print("RUNNING IPHONE REFURB STRATEGY ANALYSIS...")
        print("="*60)
        
        products, stores, sales, warranty, today = load_data(PATHS)
        iphone_strategy = analyze_iphone_refurb_strategy(products, stores, sales, warranty, today, DEFAULT_PARAMS)
        
        print(f"\nFiles saved in: {OUTDIR}")
        
        if not combined.empty:
            print(f"\n📦 REFURBISH INVENTORY DISTRIBUTION PLAN (Target Date: September 20, 2024)")
            print("="*80)
            
            # Filter for items that need restocking
            restock_needed = combined[combined["final_units_to_prestore"] > 0].copy()
            
            if not restock_needed.empty:
                # Country-level summary
                country_summary = (restock_needed
                                 .groupby("country")
                                 .agg(total_units=("final_units_to_prestore", "sum"),
                                      stores_count=("store_name", "nunique"),
                                      products_count=("product_name", "nunique"))
                                 .reset_index()
                                 .sort_values("total_units", ascending=False))
                
                print("\n🌍 COUNTRY-LEVEL ALLOCATION:")
                print("-" * 50)
                for _, row in country_summary.iterrows():
                    print(f"{row['country']:15} → {row['total_units']:4d} units "
                          f"({row['stores_count']} stores, {row['products_count']} products)")
                
                total_global_units = country_summary["total_units"].sum()
                print(f"\n🎯 TOTAL GLOBAL ALLOCATION: {total_global_units:,} refurb units")
                
                # High-priority stores (>50 units needed)
                high_priority = (restock_needed
                               .groupby(["country", "store_name"])
                               .agg(store_total=("final_units_to_prestore", "sum"))
                               .reset_index()
                               .query("store_total >= 50")
                               .sort_values("store_total", ascending=False))
                
                if not high_priority.empty:
                    print(f"\n🚨 HIGH-PRIORITY STORES (50+ units needed):")
                    print("-" * 50)
                    for _, row in high_priority.iterrows():
                        print(f"{row['country']} - {row['store_name']:30} → {row['store_total']:3d} units")
                
                # Product category breakdown
                if "category_id" in restock_needed.columns:
                    category_breakdown = (restock_needed
                                        .groupby("category_id")
                                        .agg(units=("final_units_to_prestore", "sum"))
                                        .reset_index()
                                        .sort_values("units", ascending=False))
                    
                    print(f"\n📱 PRODUCT CATEGORY BREAKDOWN:")
                    print("-" * 50)
                    for _, row in category_breakdown.iterrows():
                        pct = (row['units'] / total_global_units) * 100
                        print(f"{row['category_id']:15} → {row['units']:4d} units ({pct:.1f}%)")
                
                # Urgent shipments (need to ship by latest_stock_date)
                from datetime import datetime
                target_date = datetime(2024, 9, 20).date()
                
                urgent_mask = pd.to_datetime(restock_needed["latest_stock_date"]).dt.date <= target_date
                urgent_shipments = restock_needed[urgent_mask].copy()
                
                if not urgent_shipments.empty:
                    urgent_summary = (urgent_shipments
                                    .groupby(["country", "store_name"])
                                    .agg(urgent_units=("final_units_to_prestore", "sum"),
                                         ship_by=("latest_stock_date", "max"))
                                    .reset_index()
                                    .sort_values("urgent_units", ascending=False))
                    
                    print(f"\n⚡ URGENT SHIPMENTS (Ship by Sept 20):")
                    print("-" * 50)
                    for _, row in urgent_summary.head(10).iterrows():
                        print(f"{row['country']} - {row['store_name']:25} → {row['urgent_units']:3d} units "
                              f"(by {row['ship_by']})")
                    
                    if len(urgent_summary) > 10:
                        remaining_urgent = urgent_summary.iloc[10:]["urgent_units"].sum()
                        print(f"... and {len(urgent_summary)-10} more stores needing {remaining_urgent} units")
                
                print(f"\n📊 DISTRIBUTION LOGISTICS SUMMARY:")
                print("-" * 50)
                print(f"• Total shipments needed: {len(restock_needed):,} store×product combinations")
                print(f"• Countries to serve: {restock_needed['country'].nunique()}")
                print(f"• Stores requiring stock: {restock_needed['store_name'].nunique()}")
                print(f"• Unique products: {restock_needed['product_name'].nunique()}")
                
            else:
                print("✅ No refurbish inventory needed - all stores sufficiently stocked!")
        
    except Exception as e:
        print(f"Execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
Generating hotspot analysis...
Generating MVP forecast...
Generating PRO forecast...
Blending forecasts and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)
Forecast generation completed successfully!

RUNNING IPHONE REFURB STRATEGY ANALYSIS...
Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
Execution failed: name 'analyze_iphone_refurb_strategy' is not defined


Traceback (most recent call last):
  File "/var/folders/zt/vccr42bs5db2sjm4g8nyfcyr0000gn/T/ipykernel_94038/2056235672.py", line 653, in <module>
    iphone_strategy = analyze_iphone_refurb_strategy(products, stores, sales, warranty, today, DEFAULT_PARAMS)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'analyze_iphone_refurb_strategy' is not defined


In [4]:
# qc_forecast_playbook.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb (replacement) units
# to pre-stock for the next H days, using MVP (moving average) and PRO
# (lifecycle bucket hazard) approaches.
#
# Inputs (CSV with headers; date columns can be YYYY-MM-DD):
#   products.csv: product_id, product_name, category_id, launch_date, price (optional)
#   stores.csv:   store_id, store_name, city, country
#   sales.csv:    sale_id, sale_date, store_id, product_id, quantity
#   warranty.csv: claim_id, claim_date, sale_id, repair_status (optional)
#
# Outputs (CSV):
#   qc_hotspots.csv            - recent 180d risk ranking by claim probability (store×product)
#   qc_mvp_forecast.csv        - MVP forecast (moving average + safety stock)
#   qc_pro_forecast.csv        - PRO forecast (bucket hazard × installed base)
#   qc_final_forecast.csv      - blended final recommendation with dates
#
# Usage:
#   1) Edit the PATHS dict in __main__ to point to your CSVs.
#   2) Adjust parameters in PARAMS as needed.
#   3) Run:  python qc_forecast_playbook.py
#
# Requires: pandas, numpy (no external ML libs).
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict, Tuple, Optional

# ---------------------------
# Parameters
# ---------------------------
DEFAULT_PARAMS = {
    "HORIZON_DAYS": 30,          # planning horizon window
    "K_MONTHS_MA": 3,            # moving average window (months)
    "LEAD_TIME_DAYS": 14,        # replenishment lead time
    "SERVICE_Z": 1.65,           # 95% service level (z=1.65), 99% -> 2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# hotspot ranking lookback
    "BUCKET_DAYS": [0, 180, 360],# lifecycle buckets: [0-180], [181-360], 361+
    "BLEND_WEIGHT_PRO": 0.7,     # final blend: 70% PRO + 30% MVP (tune as needed)
    "MIN_HISTORY_CLAIMS": 30     # if product total claims < this, treat as sparse history
}


# ---------------------------
# Utilities
# ---------------------------
def _to_date(s):
    """Convert series to date, handling errors gracefully"""
    return pd.to_datetime(s, errors="coerce").dt.date

def _days_between(a, b):
    """Calculate days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def _safe_div(a, b):
    """Safe division avoiding division by zero"""
    return np.where(b == 0, 0.0, a / b)

def _ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

def _validate_file_exists(filepath: str) -> None:
    """Validate that file exists"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

def _validate_required_columns(df: pd.DataFrame, required_cols: list, filename: str) -> None:
    """Validate that required columns exist in dataframe"""
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in {filename}: {missing}")


# ---------------------------
# Loading / Cleaning
# ---------------------------
def load_data(paths: Dict[str, str]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, datetime.date]:
    """Load and validate data from CSV files"""
    
    # Validate files exist
    for name, path in paths.items():
        _validate_file_exists(path)
    
    try:
        # Load data
        products = pd.read_csv(paths["products"])
        stores = pd.read_csv(paths["stores"])
        sales = pd.read_csv(paths["sales"])
        warranty = pd.read_csv(paths["warranty"])
        
        # Validate required columns
        _validate_required_columns(products, ["product_id", "product_name", "category_id"], "products.csv")
        _validate_required_columns(stores, ["store_id", "store_name", "city", "country"], "stores.csv")
        _validate_required_columns(sales, ["sale_id", "sale_date", "store_id", "product_id", "quantity"], "sales.csv")
        _validate_required_columns(warranty, ["claim_id", "claim_date", "sale_id"], "warranty.csv")
        
        # Convert dates
        if "launch_date" in products.columns:
            products["launch_date"] = _to_date(products["launch_date"])
        sales["sale_date"] = _to_date(sales["sale_date"])
        warranty["claim_date"] = _to_date(warranty["claim_date"])
        
        # Basic cleaning
        sales = sales[sales["quantity"] > 0].copy()
        
        # Remove rows with invalid dates
        sales = sales.dropna(subset=["sale_date"])
        warranty = warranty.dropna(subset=["claim_date"])
        
        # Deduplicate
        sales = sales.drop_duplicates(subset=["sale_id"])
        warranty = warranty.drop_duplicates(subset=["claim_id"])
        
        # Calculate TODAY from actual data
        today = max(
            sales["sale_date"].max() if not sales.empty else datetime.now().date(),
            warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
        )
        
        print(f"Data loaded successfully. TODAY calculated as: {today}")
        print(f"Sales records: {len(sales)}, Warranty claims: {len(warranty)}")
        
        return products, stores, sales, warranty, today
        
    except Exception as e:
        raise ValueError(f"Error loading data: {str(e)}")


# ---------------------------
# Joins
# ---------------------------
def join_sales_warranty(products, stores, sales, warranty):
    """Join warranty claims with sales and product/store info"""
    ws = (warranty
          .merge(sales[["sale_id", "sale_date", "store_id", "product_id", "quantity"]], 
                 on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                 on="product_id", how="left")
          .merge(stores[["store_id", "store_name", "city", "country"]], 
                 on="store_id", how="left")
         )
    return ws


# ---------------------------
# Hotspot ranking (recent 180d)
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, lookback_days=180):
    """Build hotspot ranking based on recent claim probability"""
    cutoff = today - timedelta(days=lookback_days)
    
    # Recent sales and warranty claims
    s_recent = sales[sales["sale_date"] >= cutoff].copy()
    w_recent = warranty[warranty["claim_date"] >= cutoff].copy()
    
    if s_recent.empty:
        print("Warning: No recent sales data found for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales
    base = (s_recent
            .merge(products[["product_id", "product_name"]], on="product_id", how="left")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Aggregate claims
    claims = pd.DataFrame()
    if not w_recent.empty:
        claims = (w_recent
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .merge(products[["product_id", "product_name"]], on="product_id", how="left")
                  .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
                  .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)
                  .agg(claims=("claim_id", "count"))
                  .reset_index())
    
    # Merge and calculate probabilities
    if claims.empty:
        base["claims"] = 0
        df = base
    else:
        df = base.merge(claims, on=["country", "store_name", "product_id", "product_name"], how="left")
        df["claims"] = df["claims"].fillna(0).astype(int)
    
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    df = df.sort_values(["risk_rank", "country", "store_name", "product_name"])
    
    return df


# ---------------------------
# Installed base & monthly claims
# ---------------------------
def build_installed_base(stores, products, sales, warranty):
    """Calculate installed base (sold - claimed) per store×product"""
    # Cumulative sales
    sold = (sales
            .groupby(["store_id", "product_id"], as_index=False)
            .agg(units_sold=("quantity", "sum")))
    
    # Cumulative claims
    claims = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    if not warranty.empty:
        claims = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id", how="left")
                  .groupby(["store_id", "product_id"], as_index=False)
                  .agg(units_claimed=("claim_id", "count")))
    
    # Merge and calculate installed base
    base = sold.merge(claims, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0).astype(int)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id", how="left")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], 
                   on="product_id", how="left"))
    
    return base[["country", "store_name", "product_id", "product_name", 
                 "category_id", "launch_date", "installed_base"]]


def build_claims_monthly(products, stores, sales, warranty):
    """Build monthly claims aggregation"""
    if warranty.empty:
        return pd.DataFrame()
    
    ws = join_sales_warranty(products, stores, sales, warranty)
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    grp = (ws.groupby(["country", "store_name", "product_id", "product_name", "month"], dropna=False)
             .agg(claims=("claim_id", "count")).reset_index())
    
    return grp


# ---------------------------
# MVP forecast (moving average + safety)
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Generate MVP forecast using moving average approach"""
    horizon_days = params["HORIZON_DAYS"]
    k_months = params["K_MONTHS_MA"]
    z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    
    cm = build_claims_monthly(products, stores, sales, warranty)
    
    if cm.empty:
        print("Warning: No claims data available for MVP forecast")
        return pd.DataFrame()
    
    # Keep last k months for stats
    cutoff_period = pd.Timestamp(today).to_period("M") - k_months
    cm["month"] = pd.to_datetime(cm["month"].astype(str))
    cm["month_period"] = cm["month"].dt.to_period("M")
    recent = cm[cm["month_period"] >= cutoff_period]
    
    if recent.empty:
        print("Warning: No recent claims data for MVP forecast")
        return pd.DataFrame()
    
    # Calculate statistics
    stats = (recent
             .groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
             .agg(ma_k="mean", sd_k="std").reset_index())
    stats["sd_k"] = stats["sd_k"].fillna(0.0)
    
    # Convert monthly to horizon days
    stats["demand_horizon"] = _ensure_nonneg_int(np.ceil(stats["ma_k"] * (horizon_days / 30.0)))
    stats["safety_stock"] = _ensure_nonneg_int(np.ceil(z * stats["sd_k"] * np.sqrt(lead_time / 30.0)))
    stats["units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    stats["horizon_end"] = horizon_end
    stats["latest_stock_date"] = latest_stock_date
    
    # Rename columns and sort
    out = stats.rename(columns={
        "ma_k": "mvp_ma_month",
        "sd_k": "mvp_sd_month",
        "demand_horizon": "mvp_demand_horizon",
        "safety_stock": "mvp_safety",
        "units_to_prestore": "mvp_units_to_prestore"
    })
    
    out = out.sort_values(["mvp_units_to_prestore", "country", "store_name", "product_name"], 
                         ascending=[False, True, True, True])
    
    return out


# ---------------------------
# PRO forecast (bucket hazard × installed base)
# ---------------------------
def build_product_claim_prob(products, stores, sales, warranty):
    """Calculate product-level claim probability"""
    # Total sales per product
    sold = (sales.groupby(["product_id"], as_index=False)
                 .agg(units_sold=("quantity", "sum")))
    
    # Total claims per product
    claims = pd.DataFrame(columns=["product_id", "claims"])
    if not warranty.empty:
        claims = (warranty.merge(sales[["sale_id", "product_id"]], on="sale_id", how="left")
                         .groupby(["product_id"], as_index=False)
                         .agg(claims=("claim_id", "count")))
    
    # Merge and calculate probability
    df = sold.merge(claims, on="product_id", how="left")
    df["claims"] = df["claims"].fillna(0).astype(int)
    df["claim_prob"] = _safe_div(df["claims"], df["units_sold"])
    
    # Add product info
    df = df.merge(products[["product_id", "product_name", "category_id"]], on="product_id", how="left")
    
    return df[["product_id", "product_name", "category_id", "claim_prob", "claims", "units_sold"]]


def build_lifecycle_bucket_claims(products, sales, warranty, params):
    """Count claims by lifecycle bucket per product"""
    if warranty.empty:
        return pd.DataFrame()
    
    buckets = params["BUCKET_DAYS"]
    ws = (warranty
          .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id", how="left")
          .merge(products[["product_id", "product_name", "launch_date"]], on="product_id", how="left"))
    
    ws["age_days"] = _days_between(ws["claim_date"], ws["launch_date"])
    
    def bucket_label(d):
        if pd.isna(d):
            return "unknown"
        d = int(d)
        if d < 0:
            return "prelaunch"
        if d <= buckets[1]:    # 0-180
            return "0-6"
        if d <= buckets[2]:    # 181-360
            return "7-12"
        return "13+"
    
    ws["bucket"] = ws["age_days"].apply(bucket_label)
    c = (ws.groupby(["product_id", "bucket"], dropna=False)
           .agg(claims=("claim_id", "count")).reset_index())
    
    return c


def pro_forecast(products, stores, sales, warranty, today, params):
    """Generate PRO forecast using lifecycle bucket approach"""
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"]
    z = params["SERVICE_Z"]
    buckets = params["BUCKET_DAYS"]
    min_hist = params["MIN_HISTORY_CLAIMS"]
    k_months = params["K_MONTHS_MA"]
    
    # Get installed base
    base = build_installed_base(stores, products, sales, warranty)
    
    if base.empty:
        print("Warning: No installed base data for PRO forecast")
        return pd.DataFrame()
    
    # Product-level probabilities
    prod_prob = build_product_claim_prob(products, stores, sales, warranty)
    bucket_claims = build_lifecycle_bucket_claims(products, sales, warranty, params)
    
    # Handle empty bucket claims
    if bucket_claims.empty:
        # Create dummy bucket data
        unique_products = prod_prob["product_id"].unique()
        bucket_data = []
        for pid in unique_products:
            for bucket in ["0-6", "7-12", "13+"]:
                bucket_data.append({"product_id": pid, "bucket": bucket, "claims": 0})
        bucket_claims = pd.DataFrame(bucket_data)
    
    # Pivot bucket claims
    pivot = bucket_claims.pivot_table(index="product_id", columns="bucket", values="claims", 
                                     aggfunc="sum", fill_value=0).reset_index()
    
    # Merge with product probabilities
    prod = prod_prob.merge(pivot, on="product_id", how="left").fillna(0)
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in prod.columns:
            prod[b] = 0
    
    prod["total_claims_hist"] = prod[["0-6", "7-12", "13+"]].sum(axis=1)
    
    # Calculate monthly rates per bucket
    def compute_monthly_rates(row):
        total = row["total_claims_hist"]
        cp = row["claim_prob"]
        
        # Calculate shares
        if total > 0:
            s0 = row["0-6"] / total
            s1 = row["7-12"] / total
            s2 = row["13+"] / total
        else:
            s0 = s1 = s2 = 1/3.0
        
        # Monthly rates
        r0 = cp * s0 / 6.0    # 0-6 months
        r1 = cp * s1 / 6.0    # 7-12 months
        r2 = cp * s2 / 999.0  # 13+ months (very low rate)
        
        return pd.Series({"rate_0_6": r0, "rate_7_12": r1, "rate_13p": r2})
    
    rates = prod.apply(compute_monthly_rates, axis=1)
    prod = pd.concat([prod, rates], axis=1)
    
    # Determine current age bucket for each store×product
    base["age_days_now"] = _days_between(today, base["launch_date"])
    
    def current_bucket(d):
        if pd.isna(d):
            return "13+"
        d = int(d)
        if d <= buckets[1]:
            return "0-6"
        if d <= buckets[2]:
            return "7-12"
        return "13+"
    
    base["current_bucket"] = base["age_days_now"].apply(current_bucket)
    
    # Aggregate exposure by bucket
    exposure = (base
                .groupby(["country", "store_name", "product_id", "product_name", "current_bucket"], 
                        dropna=False)
                .agg(units=("installed_base", "sum"))
                .reset_index())
    
    # Pivot exposure
    expo_pivot = (exposure
                  .pivot_table(index=["country", "store_name", "product_id", "product_name"],
                              columns="current_bucket", values="units", aggfunc="sum", fill_value=0)
                  .reset_index())
    
    # Ensure all bucket columns exist
    for b in ["0-6", "7-12", "13+"]:
        if b not in expo_pivot.columns:
            expo_pivot[b] = 0
    
    # Merge with rates
    expo_pivot = expo_pivot.merge(prod[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims_hist"]], 
                                 on="product_id", how="left")
    
    # Handle sparse history
    sparse_mask = expo_pivot["total_claims_hist"].fillna(0) < min_hist
    for col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        expo_pivot.loc[sparse_mask, col] = expo_pivot.loc[sparse_mask, col] * 1.25
    
    # Calculate expected claims
    factor = horizon_days / 30.0
    expo_pivot["expected_claims"] = (
        expo_pivot["0-6"] * (expo_pivot["rate_0_6"] * factor) +
        expo_pivot["7-12"] * (expo_pivot["rate_7_12"] * factor) +
        expo_pivot["13+"] * (expo_pivot["rate_13p"] * factor)
    )
    
    # Safety stock calculation
    cm = build_claims_monthly(products, stores, sales, warranty)
    if not cm.empty:
        cutoff_period = pd.Timestamp(today).to_period("M") - k_months
        cm["month"] = pd.to_datetime(cm["month"].astype(str))
        cm["month_period"] = cm["month"].dt.to_period("M")
        recent = cm[cm["month_period"] >= cutoff_period]
        
        sd = (recent.groupby(["country", "store_name", "product_id", "product_name"], dropna=False)["claims"]
                    .std().reset_index().rename(columns={"claims": "sd_month"}))
        sd["sd_month"] = sd["sd_month"].fillna(0.0)
    else:
        sd = pd.DataFrame(columns=["country", "store_name", "product_id", "product_name", "sd_month"])
    
    # Merge safety stock data
    out = expo_pivot.merge(sd, on=["country", "store_name", "product_id", "product_name"], how="left")
    out["sd_month"] = out["sd_month"].fillna(0.0)
    
    # Calculate final values
    out["safety_stock"] = _ensure_nonneg_int(np.ceil(z * out["sd_month"] * np.sqrt(lead_time / 30.0)))
    out["expected_claims"] = _ensure_nonneg_int(np.ceil(out["expected_claims"]))
    out["pro_units_to_prestore"] = out["expected_claims"] + out["safety_stock"]
    
    # Add dates
    horizon_end = today + timedelta(days=horizon_days)
    latest_stock_date = today + timedelta(days=horizon_days - lead_time)
    out["horizon_end"] = horizon_end
    out["latest_stock_date"] = latest_stock_date
    
    return out[["country", "store_name", "product_id", "product_name",
                "expected_claims", "safety_stock", "pro_units_to_prestore",
                "horizon_end", "latest_stock_date"]]


# ---------------------------
# Final blend & save
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save results"""
    w = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty dataframes
    if mvp.empty and pro.empty:
        print("Warning: Both MVP and PRO forecasts are empty")
        combined = pd.DataFrame()
    elif mvp.empty:
        combined = pro.copy()
        combined["final_units_to_prestore"] = combined["pro_units_to_prestore"]
    elif pro.empty:
        combined = mvp.copy()
        combined["final_units_to_prestore"] = combined["mvp_units_to_prestore"]
    else:
        # Merge forecasts
        combined = pd.merge(pro, mvp, on=keys, how="outer")
        
        # Fill NaNs
        for col in ["pro_units_to_prestore", "mvp_units_to_prestore", "expected_claims", 
                   "mvp_demand_horizon", "mvp_safety"]:
            if col in combined.columns:
                combined[col] = combined[col].fillna(0)
        
        # Blend forecasts
        combined["final_units_to_prestore"] = _ensure_nonneg_int(
            np.ceil(w * combined["pro_units_to_prestore"] + (1-w) * combined["mvp_units_to_prestore"])
        )
        
        # Handle date columns
        if "horizon_end_x" in combined.columns and "horizon_end_y" in combined.columns:
            combined["horizon_end"] = combined["horizon_end_x"].fillna(combined["horizon_end_y"])
            combined["latest_stock_date"] = combined["latest_stock_date_x"].fillna(combined["latest_stock_date_y"])
            drop_cols = [c for c in combined.columns if c.endswith("_x") or c.endswith("_y")]
            combined = combined.drop(columns=drop_cols)
    
    # Add hotspot data if available
    if hotspots is not None and not hotspots.empty and not combined.empty:
        hz = hotspots[["country", "store_name", "product_id", "product_name", 
                      "claim_probability_pct", "risk_rank"]].copy()
        combined = combined.merge(hz, on=keys, how="left")
    
    # Sort results
    if not combined.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in combined.columns:
            sort_cols.append("risk_rank")
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False] + [True] * (len(sort_cols) - 1)
        combined = combined.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if hotspots is not None and not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
        
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
        
        if not combined.empty:
            combined.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(combined)} rows)")
        else:
            print("Warning: No final forecast data to save")
            
    except Exception as e:
        print(f"Error saving files: {str(e)}")
    
    return combined


# ---------------------------
# Main
# ---------------------------
def run(paths: Dict[str, str], outdir: str = ".", params: Optional[Dict] = None):
    """Main execution function"""
    if params is None:
        params = DEFAULT_PARAMS
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_data(paths)
        
        # Generate hotspots
        print("Generating hotspot analysis...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, 
                                 lookback_days=params["LOOKBACK_DAYS_HOTSPOT"])
        
        # Generate forecasts
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # Blend and save
        print("Blending forecasts and saving results...")
        combined = blend_and_save(hotspots, mvp, pro, params, outdir=outdir)
        
        print("Forecast generation completed successfully!")
        return hotspots, mvp, pro, combined
        
    except Exception as e:
        print(f"Error in forecast generation: {str(e)}")
        raise


if __name__ == "__main__":
    PATHS = {
        "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
        "stores":   "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
        "sales":    "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
        "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
    }
    
    # Optional: set output directory
    OUTDIR = "."
    
    # Run with error handling
    try:
        hotspots, mvp, pro, combined = run(PATHS, outdir=OUTDIR, params=DEFAULT_PARAMS)
        
        # Load data again for iPhone analysis
        print("\n" + "="*60)
        print("RUNNING IPHONE REFURB STRATEGY ANALYSIS...")
        print("="*60)
        
        try:
            products, stores, sales, warranty, today = load_data(PATHS)
            iphone_strategy = analyze_iphone_refurb_strategy(products, stores, sales, warranty, today, DEFAULT_PARAMS)
        except Exception as e:
            print(f"iPhone analysis failed: {str(e)}")
            print("Continuing with main forecast results...")
        
        print(f"\nFiles saved in: {OUTDIR}")
        
        if not combined.empty:
            print(f"\n📦 REFURBISH INVENTORY DISTRIBUTION PLAN (Target Date: September 20, 2024)")
            print("="*80)
            
            # Filter for items that need restocking
            restock_needed = combined[combined["final_units_to_prestore"] > 0].copy()
            
            if not restock_needed.empty:
                # Country-level summary
                country_summary = (restock_needed
                                 .groupby("country")
                                 .agg(total_units=("final_units_to_prestore", "sum"),
                                      stores_count=("store_name", "nunique"),
                                      products_count=("product_name", "nunique"))
                                 .reset_index()
                                 .sort_values("total_units", ascending=False))
                
                print("\n🌍 COUNTRY-LEVEL ALLOCATION:")
                print("-" * 50)
                for _, row in country_summary.iterrows():
                    print(f"{row['country']:15} → {row['total_units']:4d} units "
                          f"({row['stores_count']} stores, {row['products_count']} products)")
                
                total_global_units = country_summary["total_units"].sum()
                print(f"\n🎯 TOTAL GLOBAL ALLOCATION: {total_global_units:,} refurb units")
                
                # High-priority stores (>50 units needed)
                high_priority = (restock_needed
                               .groupby(["country", "store_name"])
                               .agg(store_total=("final_units_to_prestore", "sum"))
                               .reset_index()
                               .query("store_total >= 50")
                               .sort_values("store_total", ascending=False))
                
                if not high_priority.empty:
                    print(f"\n🚨 HIGH-PRIORITY STORES (50+ units needed):")
                    print("-" * 50)
                    for _, row in high_priority.iterrows():
                        print(f"{row['country']} - {row['store_name']:30} → {row['store_total']:3d} units")
                
                # Product category breakdown
                if "category_id" in restock_needed.columns:
                    category_breakdown = (restock_needed
                                        .groupby("category_id")
                                        .agg(units=("final_units_to_prestore", "sum"))
                                        .reset_index()
                                        .sort_values("units", ascending=False))
                    
                    print(f"\n📱 PRODUCT CATEGORY BREAKDOWN:")
                    print("-" * 50)
                    for _, row in category_breakdown.iterrows():
                        pct = (row['units'] / total_global_units) * 100
                        print(f"{row['category_id']:15} → {row['units']:4d} units ({pct:.1f}%)")
                
                # Urgent shipments (need to ship by latest_stock_date)
                from datetime import datetime
                target_date = datetime(2024, 9, 20).date()
                
                urgent_mask = pd.to_datetime(restock_needed["latest_stock_date"]).dt.date <= target_date
                urgent_shipments = restock_needed[urgent_mask].copy()
                
                if not urgent_shipments.empty:
                    urgent_summary = (urgent_shipments
                                    .groupby(["country", "store_name"])
                                    .agg(urgent_units=("final_units_to_prestore", "sum"),
                                         ship_by=("latest_stock_date", "max"))
                                    .reset_index()
                                    .sort_values("urgent_units", ascending=False))
                    
                    print(f"\n⚡ URGENT SHIPMENTS (Ship by Sept 20):")
                    print("-" * 50)
                    for _, row in urgent_summary.head(10).iterrows():
                        print(f"{row['country']} - {row['store_name']:25} → {row['urgent_units']:3d} units "
                              f"(by {row['ship_by']})")
                    
                    if len(urgent_summary) > 10:
                        remaining_urgent = urgent_summary.iloc[10:]["urgent_units"].sum()
                        print(f"... and {len(urgent_summary)-10} more stores needing {remaining_urgent} units")
                
                print(f"\n📊 DISTRIBUTION LOGISTICS SUMMARY:")
                print("-" * 50)
                print(f"• Total shipments needed: {len(restock_needed):,} store×product combinations")
                print(f"• Countries to serve: {restock_needed['country'].nunique()}")
                print(f"• Stores requiring stock: {restock_needed['store_name'].nunique()}")
                print(f"• Unique products: {restock_needed['product_name'].nunique()}")
                
            else:
                print("✅ No refurbish inventory needed - all stores sufficiently stocked!")
        
    except Exception as e:
        print(f"Execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
Generating hotspot analysis...
Generating MVP forecast...
Generating PRO forecast...
Blending forecasts and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)
Forecast generation completed successfully!

RUNNING IPHONE REFURB STRATEGY ANALYSIS...
Data loaded successfully. TODAY calculated as: 2024-08-21
Sales records: 1040191, Warranty claims: 30836
iPhone analysis failed: name 'analyze_iphone_refurb_strategy' is not defined
Continuing with main forecast results...

Files saved in: .

📦 REFURBISH INVENTORY DISTRIBUTION PLAN (Target Date: September 20, 2024)

🌍 COUNTRY-LEVEL ALLOCATION:
--------------------------------------------------
USA             →  460 units (10 stores, 46 products)
Turkey          →  410 units (2 stores, 46 products)
Canada          →  368 units (

In [6]:
# qc_forecast_playbook.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb units
# to pre-stock for the next H days, using MVP (moving average) 
# and PRO (lifecycle bucket hazard) approaches.
#
# Usage:
#   1) Edit PATHS to point to your 4 CSV files
#   2) Adjust PARAMS if needed  
#   3) Run: python qc_forecast_playbook.py
#
# Outputs 4 CSV files:
#   - qc_hotspots.csv (risk ranking)
#   - qc_mvp_forecast.csv (moving average forecast) 
#   - qc_pro_forecast.csv (lifecycle bucket forecast)
#   - qc_final_forecast.csv (blended recommendation)
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict

# ---------------------------
# PARAMETERS (Tune as needed)
# ---------------------------
PARAMS = {
    "HORIZON_DAYS": 30,          # Planning horizon (days)
    "K_MONTHS_MA": 3,            # Moving average window (months)
    "LEAD_TIME_DAYS": 14,        # Replenishment lead time 
    "SERVICE_Z": 1.65,           # 95% service level (1.65), 99%→2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# Hotspot analysis window
    "BUCKET_DAYS": [0, 180, 360],# Lifecycle buckets: 0-180, 181-360, 361+
    "BLEND_WEIGHT_PRO": 0.7,     # Final blend: 70% PRO + 30% MVP
    "MIN_HISTORY_CLAIMS": 30     # Sparse history threshold
}

# ---------------------------
# Utilities
# ---------------------------
def to_date(s):
    """Convert to date, handle errors"""
    return pd.to_datetime(s, errors="coerce").dt.date

def days_between(a, b):
    """Days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def safe_div(a, b):
    """Safe division"""
    return np.where(b == 0, 0.0, a / b)

def ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

# ---------------------------
# Data Loading & Validation
# ---------------------------
def load_and_validate_data(paths):
    """Load and validate 4 CSV files"""
    
    # Check files exist
    for name, path in paths.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
    
    # Load data
    products = pd.read_csv(paths["products"])
    stores = pd.read_csv(paths["stores"])
    sales = pd.read_csv(paths["sales"])
    warranty = pd.read_csv(paths["warranty"])
    
    # Required columns check
    req_cols = {
        "products": ["product_id", "product_name", "category_id"],
        "stores": ["store_id", "store_name", "city", "country"], 
        "sales": ["sale_id", "sale_date", "store_id", "product_id", "quantity"],
        "warranty": ["claim_id", "claim_date", "sale_id"]
    }
    
    for name, cols in req_cols.items():
        df = locals()[name]
        missing = [c for c in cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns in {name}.csv: {missing}")
    
    # Convert dates and clean
    if "launch_date" in products.columns:
        products["launch_date"] = to_date(products["launch_date"])
    sales["sale_date"] = to_date(sales["sale_date"])
    warranty["claim_date"] = to_date(warranty["claim_date"])
    
    # Basic cleaning
    sales = sales[sales["quantity"] > 0].dropna(subset=["sale_date"])
    warranty = warranty.dropna(subset=["claim_date"])
    sales = sales.drop_duplicates(subset=["sale_id"])
    warranty = warranty.drop_duplicates(subset=["claim_id"])
    
    # Calculate TODAY
    today = max(
        sales["sale_date"].max() if not sales.empty else datetime.now().date(),
        warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
    )
    
    print(f"✓ Data loaded. Sales: {len(sales):,}, Claims: {len(warranty):,}, TODAY: {today}")
    return products, stores, sales, warranty, today

# ---------------------------
# A) Hotspot Ranking
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, params):
    """Recent claim probability ranking by store×product"""
    
    cutoff = today - timedelta(days=params["LOOKBACK_DAYS_HOTSPOT"])
    
    # Recent sales
    recent_sales = sales[sales["sale_date"] >= cutoff].copy()
    if recent_sales.empty:
        print("⚠️ No recent sales for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales by store×product
    base = (recent_sales
            .merge(products[["product_id", "product_name"]], on="product_id")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .groupby(["country", "store_name", "product_id", "product_name"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Recent claims
    recent_claims = warranty[warranty["claim_date"] >= cutoff].copy()
    if not recent_claims.empty:
        claims_agg = (recent_claims
                     .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                     .merge(products[["product_id", "product_name"]], on="product_id")
                     .merge(stores[["store_id", "store_name", "country"]], on="store_id")
                     .groupby(["country", "store_name", "product_id", "product_name"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
        
        df = base.merge(claims_agg, on=["country", "store_name", "product_id", "product_name"], how="left")
    else:
        df = base.copy()
    
    df["claims"] = df["claims"].fillna(0).astype(int)
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    
    return df.sort_values(["risk_rank", "country", "store_name", "product_name"])

# ---------------------------
# B) MVP Forecast 
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Moving average + safety stock forecast"""
    
    if warranty.empty:
        print("⚠️ No warranty data for MVP forecast")
        return pd.DataFrame()
    
    # Monthly claims aggregation
    ws = (warranty
          .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
          .merge(products[["product_id", "product_name"]], on="product_id")
          .merge(stores[["store_id", "store_name", "country"]], on="store_id"))
    
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    monthly_claims = (ws
                     .groupby(["country", "store_name", "product_id", "product_name", "month"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
    
    # Keep last K months
    cutoff_period = pd.Timestamp(today).to_period("M") - params["K_MONTHS_MA"]
    recent_claims = monthly_claims[monthly_claims["month"] >= cutoff_period]
    
    if recent_claims.empty:
        print("⚠️ No recent claims for MVP forecast")
        return pd.DataFrame()
    
    # Statistics per store×product
    stats = (recent_claims
             .groupby(["country", "store_name", "product_id", "product_name"])["claims"]
             .agg(mean_monthly="mean", std_monthly="std")
             .reset_index())
    
    stats["std_monthly"] = stats["std_monthly"].fillna(0.0)
    
    # Convert to horizon period
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"] 
    service_z = params["SERVICE_Z"]
    
    stats["demand_horizon"] = ensure_nonneg_int(np.ceil(stats["mean_monthly"] * (horizon_days / 30.0)))
    stats["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * stats["std_monthly"] * np.sqrt(lead_time / 30.0)))
    stats["mvp_units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    stats["horizon_end"] = today + timedelta(days=horizon_days)
    stats["latest_stock_date"] = today + timedelta(days=horizon_days - lead_time)
    
    return stats.sort_values("mvp_units_to_prestore", ascending=False)

# ---------------------------
# C) PRO Forecast
# ---------------------------
def calculate_installed_base(stores, products, sales, warranty):
    """Calculate installed base = sold - claimed per store×product"""
    
    # Total sales
    sold = (sales
            .groupby(["store_id", "product_id"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Total claims
    if not warranty.empty:
        claimed = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                  .groupby(["store_id", "product_id"])
                  .agg(units_claimed=("claim_id", "count"))
                  .reset_index())
    else:
        claimed = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    
    # Merge and calculate installed base
    base = sold.merge(claimed, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], on="product_id"))
    
    return base

def calculate_product_hazards(products, sales, warranty, params):
    """Calculate product-level claim probability and lifecycle bucket hazards"""
    
    # Overall product claim rates
    product_sales = sales.groupby("product_id").agg(total_sold=("quantity", "sum")).reset_index()
    
    if not warranty.empty:
        product_claims = (warranty
                         .merge(sales[["sale_id", "product_id"]], on="sale_id")
                         .groupby("product_id")
                         .agg(total_claims=("claim_id", "count"))
                         .reset_index())
        
        product_rates = product_sales.merge(product_claims, on="product_id", how="left")
        product_rates["total_claims"] = product_rates["total_claims"].fillna(0)
        product_rates["claim_prob"] = safe_div(product_rates["total_claims"], product_rates["total_sold"])
    else:
        product_rates = product_sales.copy()
        product_rates["total_claims"] = 0
        product_rates["claim_prob"] = 0.0
    
    # Lifecycle bucket analysis
    if not warranty.empty:
        bucket_claims = (warranty
                        .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id")
                        .merge(products[["product_id", "launch_date"]], on="product_id"))
        
        bucket_claims["age_days"] = days_between(bucket_claims["claim_date"], bucket_claims["launch_date"])
        
        def assign_bucket(days):
            if pd.isna(days) or days < 0:
                return "unknown"
            elif days <= params["BUCKET_DAYS"][1]:  # 0-180
                return "bucket_0_6"
            elif days <= params["BUCKET_DAYS"][2]:  # 181-360
                return "bucket_7_12" 
            else:
                return "bucket_13p"
        
        bucket_claims["bucket"] = bucket_claims["age_days"].apply(assign_bucket)
        
        bucket_pivot = (bucket_claims
                       .groupby(["product_id", "bucket"])
                       .agg(claims=("claim_id", "count"))
                       .reset_index()
                       .pivot_table(index="product_id", columns="bucket", values="claims", fill_value=0)
                       .reset_index())
        
        # Merge with product rates
        hazards = product_rates.merge(bucket_pivot, on="product_id", how="left").fillna(0)
    else:
        hazards = product_rates.copy()
        for bucket in ["bucket_0_6", "bucket_7_12", "bucket_13p"]:
            hazards[bucket] = 0
    
    # Calculate bucket shares and monthly rates
    bucket_cols = ["bucket_0_6", "bucket_7_12", "bucket_13p"]
    hazards["total_bucket_claims"] = hazards[bucket_cols].sum(axis=1)
    
    for bucket in bucket_cols:
        if hazards["total_bucket_claims"].sum() > 0:
            hazards[f"{bucket}_share"] = safe_div(hazards[bucket], hazards["total_bucket_claims"])
        else:
            hazards[f"{bucket}_share"] = 1/3  # Equal split if no history
    
    # Monthly hazard rates (claim_prob × bucket_share / months_in_bucket)
    hazards["rate_0_6"] = hazards["claim_prob"] * hazards["bucket_0_6_share"] / 6.0
    hazards["rate_7_12"] = hazards["claim_prob"] * hazards["bucket_7_12_share"] / 6.0  
    hazards["rate_13p"] = hazards["claim_prob"] * hazards["bucket_13p_share"] / 999.0  # Very low rate
    
    return hazards

def pro_forecast(products, stores, sales, warranty, today, params):
    """Lifecycle bucket hazard × installed base forecast"""
    
    # Get installed base and hazard rates
    installed_base = calculate_installed_base(stores, products, sales, warranty)
    hazards = calculate_product_hazards(products, sales, warranty, params)
    
    if installed_base.empty:
        print("⚠️ No installed base for PRO forecast")
        return pd.DataFrame()
    
    # Determine current age bucket for each store×product
    installed_base["age_days"] = days_between(today, installed_base["launch_date"])
    
    def current_bucket(days):
        if pd.isna(days) or days < 0:
            return "bucket_13p"
        elif days <= params["BUCKET_DAYS"][1]:
            return "bucket_0_6"
        elif days <= params["BUCKET_DAYS"][2]:
            return "bucket_7_12"
        else:
            return "bucket_13p"
    
    installed_base["current_bucket"] = installed_base["age_days"].apply(current_bucket)
    
    # Merge with hazard rates
    forecast = installed_base.merge(hazards[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims"]], on="product_id")
    
    # Apply conservative factor for sparse history
    sparse_mask = forecast["total_claims"] < params["MIN_HISTORY_CLAIMS"]
    for rate_col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        forecast.loc[sparse_mask, rate_col] *= 1.25
    
    # Calculate expected claims for horizon period
    horizon_days = params["HORIZON_DAYS"]
    horizon_factor = horizon_days / 30.0
    
    forecast["rate"] = np.where(forecast["current_bucket"] == "bucket_0_6", forecast["rate_0_6"],
                       np.where(forecast["current_bucket"] == "bucket_7_12", forecast["rate_7_12"], 
                               forecast["rate_13p"]))
    
    forecast["expected_claims"] = ensure_nonneg_int(np.ceil(forecast["installed_base"] * forecast["rate"] * horizon_factor))
    
    # Safety stock (simplified - using global variance)
    global_std = 0.5  # Conservative assumption
    service_z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    forecast["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * global_std * np.sqrt(lead_time / 30.0)))
    
    forecast["pro_units_to_prestore"] = forecast["expected_claims"] + forecast["safety_stock"]
    
    # Add dates
    forecast["horizon_end"] = today + timedelta(days=horizon_days)
    forecast["latest_stock_date"] = today + timedelta(days=horizon_days - params["LEAD_TIME_DAYS"])
    
    return forecast[["country", "store_name", "product_id", "product_name", 
                    "expected_claims", "safety_stock", "pro_units_to_prestore",
                    "horizon_end", "latest_stock_date"]].sort_values("pro_units_to_prestore", ascending=False)

# ---------------------------
# D) Final Blending & Output
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save all outputs"""
    
    w_pro = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty forecasts
    if mvp.empty and pro.empty:
        print("⚠️ Both forecasts empty")
        final = pd.DataFrame()
    elif mvp.empty:
        final = pro.copy()
        final["final_units_to_prestore"] = final["pro_units_to_prestore"]
    elif pro.empty:
        final = mvp.copy() 
        final["final_units_to_prestore"] = final["mvp_units_to_prestore"]
    else:
        # Merge and blend
        final = pro.merge(mvp, on=keys, how="outer", suffixes=("", "_mvp"))
        final["pro_units_to_prestore"] = final["pro_units_to_prestore"].fillna(0)
        final["mvp_units_to_prestore"] = final["mvp_units_to_prestore"].fillna(0)
        
        final["final_units_to_prestore"] = ensure_nonneg_int(
            w_pro * final["pro_units_to_prestore"] + (1-w_pro) * final["mvp_units_to_prestore"]
        )
        
        # Use PRO dates if available
        final["horizon_end"] = final["horizon_end"].fillna(final.get("horizon_end_mvp"))
        final["latest_stock_date"] = final["latest_stock_date"].fillna(final.get("latest_stock_date_mvp"))
    
    # Add hotspot ranking
    if not hotspots.empty and not final.empty:
        final = final.merge(hotspots[keys + ["claim_probability_pct", "risk_rank"]], on=keys, how="left")
    
    # Sort by priority
    if not final.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in final.columns:
            sort_cols.append("risk_rank") 
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False, True, True, True, True][:len(sort_cols)]
        final = final.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"✓ Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"✓ Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
            
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"✓ Saved qc_pro_forecast.csv ({len(pro)} rows)")
            
        if not final.empty:
            final.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"✓ Saved qc_final_forecast.csv ({len(final)} rows)")
        
    except Exception as e:
        print(f"❌ Save error: {e}")
    
    return final

# ---------------------------
# Main Execution
# ---------------------------
def run_forecast_pipeline(paths, outdir=".", params=None):
    """Main pipeline execution"""
    
    if params is None:
        params = PARAMS
        
    print("🚀 Starting QC Forecast Pipeline...")
    print("="*50)
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_and_validate_data(paths)
        
        # A) Hotspot analysis
        print("\n📍 Building hotspot ranking...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, params)
        
        # B) MVP forecast  
        print("📊 Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        # C) PRO forecast
        print("🎯 Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # D) Blend and save
        print("🔄 Blending and saving results...")
        final = blend_and_save(hotspots, mvp, pro, params, outdir)
        
        # Summary output
        print(f"\n📦 REFURBISH INVENTORY SUMMARY (Target: {today + timedelta(days=params['HORIZON_DAYS'])})")
        print("="*70)
        
        if not final.empty:
            restock = final[final["final_units_to_prestore"] > 0]
            
            if not restock.empty:
                # Country summary
                country_summary = (restock
                                 .groupby("country")
                                 .agg(total_units=("final_units_to_prestore", "sum"),
                                      stores=("store_name", "nunique"))
                                 .reset_index()
                                 .sort_values("total_units", ascending=False))
                
                print("🌍 COUNTRY ALLOCATION:")
                for _, row in country_summary.iterrows():
                    print(f"  {row['country']:15} → {row['total_units']:4d} units ({row['stores']} stores)")
                
                total_units = country_summary["total_units"].sum()
                total_countries = len(country_summary)
                total_stores = restock["store_name"].nunique()
                
                print(f"\n🎯 GLOBAL TOTAL: {total_units:,} units across {total_countries} countries, {total_stores} stores")
                
                # High priority stores
                high_priority = (restock
                               .groupby(["country", "store_name"])
                               .agg(store_total=("final_units_to_prestore", "sum"))
                               .reset_index()
                               .query("store_total >= 50")
                               .sort_values("store_total", ascending=False))
                
                if not high_priority.empty:
                    print(f"\n🚨 HIGH-PRIORITY STORES (≥50 units):")
                    for _, row in high_priority.head(10).iterrows():
                        print(f"  {row['country']} - {row['store_name']:30} → {row['store_total']:3d} units")
            else:
                print("✅ All stores sufficiently stocked!")
        
        print(f"\n🎉 Pipeline completed successfully!")
        print(f"📁 Results saved in: {outdir}/")
        
        return final
        
    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        raise

# ---------------------------
# Script Entry Point
# ---------------------------
if __name__ == "__main__":
    
    PATHS = {
    "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
    "stores": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
    "sales": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
    "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
}
    
    # Output directory
    OUTDIR = "."
    
    # Run pipeline
    final_forecast = run_forecast_pipeline(PATHS, outdir=OUTDIR, params=PARAMS)

🚀 Starting QC Forecast Pipeline...
✓ Data loaded. Sales: 1,040,191, Claims: 30,836, TODAY: 2024-08-21

📍 Building hotspot ranking...
📊 Generating MVP forecast...
🎯 Generating PRO forecast...
🔄 Blending and saving results...
✓ Saved qc_hotspots.csv (204 rows)
✓ Saved qc_mvp_forecast.csv (36 rows)
✓ Saved qc_pro_forecast.csv (3100 rows)
✓ Saved qc_final_forecast.csv (3100 rows)

📦 REFURBISH INVENTORY SUMMARY (Target: 2024-09-20)
🌍 COUNTRY ALLOCATION:
  USA             →  640 units (10 stores)
  Canada          →  512 units (8 stores)
  Turkey          →  268 units (2 stores)
  UK              →  204 units (7 stores)
  India           →  203 units (2 stores)
  China           →  140 units (4 stores)
  Japan           →  128 units (2 stores)
  Germany         →  122 units (2 stores)
  Australia       →  120 units (3 stores)
  France          →  116 units (2 stores)
  UAE             →   99 units (2 stores)
  Spain           →   80 units (2 stores)
  Brazil          →   80 units (2 stores)


In [10]:
# qc_forecast.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb units
# to pre-stock for the next H days, using MVP (moving average) 
# and PRO (lifecycle bucket hazard) approaches.
#
# Usage:
#   1) Edit PATHS to point to your 4 CSV files
#   2) Adjust PARAMS if needed  
#   3) Run: python qc_forecast.py
#
# Outputs 4 CSV files:
#   - qc_hotspots.csv (risk ranking)
#   - qc_mvp_forecast.csv (moving average forecast) 
#   - qc_pro_forecast.csv (lifecycle bucket forecast)
#   - qc_final_forecast.csv (blended recommendation)
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict

# ---------------------------
# PARAMETERS (Tune as needed)
# ---------------------------
PARAMS = {
    "HORIZON_DAYS": 30,          # Planning horizon (days)
    "K_MONTHS_MA": 3,            # Moving average window (months)
    "LEAD_TIME_DAYS": 14,        # Replenishment lead time 
    "SERVICE_Z": 1.65,           # 95% service level (1.65), 99%→2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# Hotspot analysis window
    "BUCKET_DAYS": [0, 180, 360],# Lifecycle buckets: 0-180, 181-360, 361+
    "BLEND_WEIGHT_PRO": 0.7,     # Final blend: 70% PRO + 30% MVP
    "MIN_HISTORY_CLAIMS": 30     # Sparse history threshold
}

# ---------------------------
# Utilities
# ---------------------------
def to_date(s):
    """Convert to date, handle errors"""
    return pd.to_datetime(s, errors="coerce").dt.date

def days_between(a, b):
    """Days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def safe_div(a, b):
    """Safe division"""
    return np.where(b == 0, 0.0, a / b)

def ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

# ---------------------------
# Data Loading & Validation
# ---------------------------
def load_and_validate_data(paths):
    """Load and validate 4 CSV files"""
    
    # Check files exist
    for name, path in paths.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
    
    # Load data
    products = pd.read_csv(paths["products"])
    stores = pd.read_csv(paths["stores"])
    sales = pd.read_csv(paths["sales"])
    warranty = pd.read_csv(paths["warranty"])
    
    # Required columns check
    req_cols = {
        "products": ["product_id", "product_name", "category_id"],
        "stores": ["store_id", "store_name", "city", "country"], 
        "sales": ["sale_id", "sale_date", "store_id", "product_id", "quantity"],
        "warranty": ["claim_id", "claim_date", "sale_id"]
    }
    
    for name, cols in req_cols.items():
        df = locals()[name]
        missing = [c for c in cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns in {name}.csv: {missing}")
    
    # Convert dates and clean
    if "launch_date" in products.columns:
        products["launch_date"] = to_date(products["launch_date"])
    sales["sale_date"] = to_date(sales["sale_date"])
    warranty["claim_date"] = to_date(warranty["claim_date"])
    
    # Basic cleaning
    sales = sales[sales["quantity"] > 0].dropna(subset=["sale_date"])
    warranty = warranty.dropna(subset=["claim_date"])
    sales = sales.drop_duplicates(subset=["sale_id"])
    warranty = warranty.drop_duplicates(subset=["claim_id"])
    
    # Calculate TODAY
    today = max(
        sales["sale_date"].max() if not sales.empty else datetime.now().date(),
        warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
    )
    
    print(f" Data loaded. Sales: {len(sales):,}, Claims: {len(warranty):,}, TODAY: {today}")
    return products, stores, sales, warranty, today

# ---------------------------
# A) Hotspot Ranking
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, params):
    """Recent claim probability ranking by store×product"""
    
    cutoff = today - timedelta(days=params["LOOKBACK_DAYS_HOTSPOT"])
    
    # Recent sales
    recent_sales = sales[sales["sale_date"] >= cutoff].copy()
    if recent_sales.empty:
        print("No recent sales for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales by store×product
    base = (recent_sales
            .merge(products[["product_id", "product_name"]], on="product_id")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .groupby(["country", "store_name", "product_id", "product_name"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Recent claims
    recent_claims = warranty[warranty["claim_date"] >= cutoff].copy()
    if not recent_claims.empty:
        claims_agg = (recent_claims
                     .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                     .merge(products[["product_id", "product_name"]], on="product_id")
                     .merge(stores[["store_id", "store_name", "country"]], on="store_id")
                     .groupby(["country", "store_name", "product_id", "product_name"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
        
        df = base.merge(claims_agg, on=["country", "store_name", "product_id", "product_name"], how="left")
    else:
        df = base.copy()
    
    df["claims"] = df["claims"].fillna(0).astype(int)
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    
    return df.sort_values(["risk_rank", "country", "store_name", "product_name"])

# ---------------------------
# B) MVP Forecast 
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Moving average + safety stock forecast"""
    
    if warranty.empty:
        print("No warranty data for MVP forecast")
        return pd.DataFrame()
    
    # Monthly claims aggregation
    ws = (warranty
          .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
          .merge(products[["product_id", "product_name"]], on="product_id")
          .merge(stores[["store_id", "store_name", "country"]], on="store_id"))
    
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    monthly_claims = (ws
                     .groupby(["country", "store_name", "product_id", "product_name", "month"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
    
    # Keep last K months
    cutoff_period = pd.Timestamp(today).to_period("M") - params["K_MONTHS_MA"]
    recent_claims = monthly_claims[monthly_claims["month"] >= cutoff_period]
    
    if recent_claims.empty:
        print("No recent claims for MVP forecast")
        return pd.DataFrame()
    
    # Statistics per store×product
    stats = (recent_claims
             .groupby(["country", "store_name", "product_id", "product_name"])["claims"]
             .agg(mean_monthly="mean", std_monthly="std")
             .reset_index())
    
    stats["std_monthly"] = stats["std_monthly"].fillna(0.0)
    
    # Convert to horizon period
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"] 
    service_z = params["SERVICE_Z"]
    
    stats["demand_horizon"] = ensure_nonneg_int(np.ceil(stats["mean_monthly"] * (horizon_days / 30.0)))
    stats["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * stats["std_monthly"] * np.sqrt(lead_time / 30.0)))
    stats["mvp_units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    stats["horizon_end"] = today + timedelta(days=horizon_days)
    stats["latest_stock_date"] = today + timedelta(days=horizon_days - lead_time)
    
    return stats.sort_values("mvp_units_to_prestore", ascending=False)

# ---------------------------
# C) PRO Forecast
# ---------------------------
def calculate_installed_base(stores, products, sales, warranty):
    """Calculate installed base = sold - claimed per store×product"""
    
    # Total sales
    sold = (sales
            .groupby(["store_id", "product_id"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Total claims
    if not warranty.empty:
        claimed = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                  .groupby(["store_id", "product_id"])
                  .agg(units_claimed=("claim_id", "count"))
                  .reset_index())
    else:
        claimed = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    
    # Merge and calculate installed base
    base = sold.merge(claimed, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], on="product_id"))
    
    return base

def calculate_product_hazards(products, sales, warranty, params):
    """Calculate product-level claim probability and lifecycle bucket hazards"""
    
    # Overall product claim rates
    product_sales = sales.groupby("product_id").agg(total_sold=("quantity", "sum")).reset_index()
    
    if not warranty.empty:
        product_claims = (warranty
                         .merge(sales[["sale_id", "product_id"]], on="sale_id")
                         .groupby("product_id")
                         .agg(total_claims=("claim_id", "count"))
                         .reset_index())
        
        product_rates = product_sales.merge(product_claims, on="product_id", how="left")
        product_rates["total_claims"] = product_rates["total_claims"].fillna(0)
        product_rates["claim_prob"] = safe_div(product_rates["total_claims"], product_rates["total_sold"])
    else:
        product_rates = product_sales.copy()
        product_rates["total_claims"] = 0
        product_rates["claim_prob"] = 0.0
    
    # Lifecycle bucket analysis
    if not warranty.empty:
        bucket_claims = (warranty
                        .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id")
                        .merge(products[["product_id", "launch_date"]], on="product_id"))
        
        bucket_claims["age_days"] = days_between(bucket_claims["claim_date"], bucket_claims["launch_date"])
        
        def assign_bucket(days):
            if pd.isna(days) or days < 0:
                return "unknown"
            elif days <= params["BUCKET_DAYS"][1]:  # 0-180
                return "bucket_0_6"
            elif days <= params["BUCKET_DAYS"][2]:  # 181-360
                return "bucket_7_12" 
            else:
                return "bucket_13p"
        
        bucket_claims["bucket"] = bucket_claims["age_days"].apply(assign_bucket)
        
        bucket_pivot = (bucket_claims
                       .groupby(["product_id", "bucket"])
                       .agg(claims=("claim_id", "count"))
                       .reset_index()
                       .pivot_table(index="product_id", columns="bucket", values="claims", fill_value=0)
                       .reset_index())
        
        # Merge with product rates
        hazards = product_rates.merge(bucket_pivot, on="product_id", how="left").fillna(0)
    else:
        hazards = product_rates.copy()
        for bucket in ["bucket_0_6", "bucket_7_12", "bucket_13p"]:
            hazards[bucket] = 0
    
    # Calculate bucket shares and monthly rates
    bucket_cols = ["bucket_0_6", "bucket_7_12", "bucket_13p"]
    hazards["total_bucket_claims"] = hazards[bucket_cols].sum(axis=1)
    
    for bucket in bucket_cols:
        if hazards["total_bucket_claims"].sum() > 0:
            hazards[f"{bucket}_share"] = safe_div(hazards[bucket], hazards["total_bucket_claims"])
        else:
            hazards[f"{bucket}_share"] = 1/3  # Equal split if no history
    
    # Monthly hazard rates (claim_prob × bucket_share / months_in_bucket)
    hazards["rate_0_6"] = hazards["claim_prob"] * hazards["bucket_0_6_share"] / 6.0
    hazards["rate_7_12"] = hazards["claim_prob"] * hazards["bucket_7_12_share"] / 6.0  
    hazards["rate_13p"] = hazards["claim_prob"] * hazards["bucket_13p_share"] / 999.0
    
    return hazards

def pro_forecast(products, stores, sales, warranty, today, params):
    """Lifecycle bucket hazard × installed base forecast"""
    
    # Get installed base and hazard rates
    installed_base = calculate_installed_base(stores, products, sales, warranty)
    hazards = calculate_product_hazards(products, sales, warranty, params)
    
    if installed_base.empty:
        print("No installed base for PRO forecast")
        return pd.DataFrame()
    
    # Determine current age bucket for each store×product
    installed_base["age_days"] = days_between(today, installed_base["launch_date"])
    
    def current_bucket(days):
        if pd.isna(days) or days < 0:
            return "bucket_13p"
        elif days <= params["BUCKET_DAYS"][1]:
            return "bucket_0_6"
        elif days <= params["BUCKET_DAYS"][2]:
            return "bucket_7_12"
        else:
            return "bucket_13p"
    
    installed_base["current_bucket"] = installed_base["age_days"].apply(current_bucket)
    
    # Merge with hazard rates
    forecast = installed_base.merge(hazards[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims"]], on="product_id")
    
    # Apply conservative factor for sparse history
    sparse_mask = forecast["total_claims"] < params["MIN_HISTORY_CLAIMS"]
    for rate_col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        forecast.loc[sparse_mask, rate_col] *= 1.25
    
    # Calculate expected claims for horizon period
    horizon_days = params["HORIZON_DAYS"]
    horizon_factor = horizon_days / 30.0
    
    forecast["rate"] = np.where(forecast["current_bucket"] == "bucket_0_6", forecast["rate_0_6"],
                       np.where(forecast["current_bucket"] == "bucket_7_12", forecast["rate_7_12"], 
                               forecast["rate_13p"]))
    
    forecast["expected_claims"] = ensure_nonneg_int(np.ceil(forecast["installed_base"] * forecast["rate"] * horizon_factor))
    
    # Safety stock (simplified - using global variance)
    global_std = 0.5  # Conservative assumption
    service_z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    forecast["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * global_std * np.sqrt(lead_time / 30.0)))
    
    forecast["pro_units_to_prestore"] = forecast["expected_claims"] + forecast["safety_stock"]
    
    # Add dates
    forecast["horizon_end"] = today + timedelta(days=horizon_days)
    forecast["latest_stock_date"] = today + timedelta(days=horizon_days - params["LEAD_TIME_DAYS"])
    
    return forecast[["country", "store_name", "product_id", "product_name", 
                    "expected_claims", "safety_stock", "pro_units_to_prestore",
                    "horizon_end", "latest_stock_date"]].sort_values("pro_units_to_prestore", ascending=False)

# ---------------------------
# D) Final Blending & Output
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save all outputs"""
    
    w_pro = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty forecasts
    if mvp.empty and pro.empty:
        print("Both forecasts empty")
        final = pd.DataFrame()
    elif mvp.empty:
        final = pro.copy()
        final["final_units_to_prestore"] = final["pro_units_to_prestore"]
    elif pro.empty:
        final = mvp.copy() 
        final["final_units_to_prestore"] = final["mvp_units_to_prestore"]
    else:
        # Merge and blend
        final = pro.merge(mvp, on=keys, how="outer", suffixes=("", "_mvp"))
        final["pro_units_to_prestore"] = final["pro_units_to_prestore"].fillna(0)
        final["mvp_units_to_prestore"] = final["mvp_units_to_prestore"].fillna(0)
        
        final["final_units_to_prestore"] = ensure_nonneg_int(
            w_pro * final["pro_units_to_prestore"] + (1-w_pro) * final["mvp_units_to_prestore"]
        )
        
        # Use PRO dates if available
        final["horizon_end"] = final["horizon_end"].fillna(final.get("horizon_end_mvp"))
        final["latest_stock_date"] = final["latest_stock_date"].fillna(final.get("latest_stock_date_mvp"))
    
    # Add hotspot ranking
    if not hotspots.empty and not final.empty:
        final = final.merge(hotspots[keys + ["claim_probability_pct", "risk_rank"]], on=keys, how="left")
    
    # Sort by priority
    if not final.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in final.columns:
            sort_cols.append("risk_rank") 
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False, True, True, True, True][:len(sort_cols)]
        final = final.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
            
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
            
        if not final.empty:
            final.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(final)} rows)")
        
    except Exception as e:
        print(f"Save error: {e}")
    
    return final

# ---------------------------
# Main Execution
# ---------------------------
def run_forecast_pipeline(paths, outdir=".", params=None):
    """Main pipeline execution"""
    
    if params is None:
        params = PARAMS
        
    print("Starting QC Forecast Pipeline...")
    print("="*50)
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_and_validate_data(paths)
        
        # A) Hotspot analysis
        print("Building hotspot ranking...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, params)
        
        # B) MVP forecast  
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        # C) PRO forecast
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # D) Blend and save
        print("Blending and saving results...")
        final = blend_and_save(hotspots, mvp, pro, params, outdir)
        
        # Detailed inventory allocation output
        print(f"\nDETAILED REFURBISH INVENTORY ALLOCATION")
        print(f"Target Date: {today + timedelta(days=params['HORIZON_DAYS'])}")
        print("="*80)
        
        if not final.empty:
            restock = final[final["final_units_to_prestore"] > 0].copy()
            
            if not restock.empty:
                # Sort by urgency (latest_stock_date) and quantity
                restock = restock.sort_values(["latest_stock_date", "final_units_to_prestore"], 
                                            ascending=[True, False])
                
                print("\nSTORE-BY-STORE ALLOCATION (Ship by Latest Stock Date):")
                print("-" * 80)
                
                current_store = ""
                store_total = 0
                
                for _, row in restock.iterrows():
                    store_key = f"{row['country']} - {row['store_name']}"
                    
                    # New store header
                    if store_key != current_store:
                        if current_store:  # Print previous store total
                            print(f"    └─ Store Total: {store_total} units\n")
                        
                        print(f"{store_key}")
                        print(f"   Ship by: {row['latest_stock_date']}")
                        current_store = store_key
                        store_total = 0
                    
                    # Product line
                    units = int(row['final_units_to_prestore'])
                    product = row['product_name'][:40]  # Truncate long names
                    risk_info = ""
                    if 'risk_rank' in row and pd.notna(row['risk_rank']):
                        risk_info = f" (Risk #{int(row['risk_rank'])})"
                    
                    print(f"   • {product:<40} → {units:3d} units{risk_info}")
                    store_total += units
                
                # Print last store total
                if current_store:
                    print(f"    └─ Store Total: {store_total} units\n")
                
                # Summary statistics
                total_units = restock["final_units_to_prestore"].sum()
                total_stores = restock["store_name"].nunique()
                total_products = restock["product_name"].nunique()
                total_countries = restock["country"].nunique()
                
                print(f"ALLOCATION SUMMARY:")
                print("-" * 40)
                print(f"Total Units to Ship:     {total_units:,}")
                print(f"Stores to Serve:         {total_stores}")
                print(f"Unique Products:         {total_products}")
                print(f"Countries:               {total_countries}")
                
                # Top products needing refurb
                top_products = (restock
                              .groupby("product_name")
                              .agg(total_needed=("final_units_to_prestore", "sum"),
                                   stores_affected=("store_name", "nunique"))
                              .reset_index()
                              .sort_values("total_needed", ascending=False)
                              .head(10))
                
                print(f"\n TOP 10 PRODUCTS NEEDING REFURB:")
                print("-" * 50)
                for _, row in top_products.iterrows():
                    product = row['product_name'][:35]
                    print(f"{product:<35} → {int(row['total_needed']):4d} units ({int(row['stores_affected'])} stores)")
                
                # Urgent shipments (next 7 days)
                urgent_date = today + timedelta(days=7)
                urgent = restock[pd.to_datetime(restock['latest_stock_date']) <= pd.to_datetime(urgent_date)]
                
                if not urgent.empty:
                    urgent_total = urgent["final_units_to_prestore"].sum()
                    urgent_stores = urgent["store_name"].nunique()
                    
                    print(f"\n⚡ URGENT SHIPMENTS (Next 7 Days):")
                    print("-" * 40)
                    print(f"Units needed ASAP:       {urgent_total:,}")
                    print(f"Stores requiring urgent: {urgent_stores}")
                    
                    # Show top urgent stores
                    urgent_by_store = (urgent
                                     .groupby(["country", "store_name"])
                                     .agg(urgent_units=("final_units_to_prestore", "sum"),
                                          ship_by=("latest_stock_date", "min"))
                                     .reset_index()
                                     .sort_values("urgent_units", ascending=False)
                                     .head(5))
                    
                    print(f"\nTop 5 Urgent Stores:")
                    for _, row in urgent_by_store.iterrows():
                        print(f"  {row['country']} - {row['store_name'][:25]:<25} → {int(row['urgent_units']):3d} units by {row['ship_by']}")
                
            else:
                print("All stores sufficiently stocked!")
        
        print(f"\n🎉 Pipeline completed successfully!")
        print(f"📁 Results saved in: {outdir}/")
        
        return final
        
    except Exception as e:
        print(f"Pipeline failed: {e}")
        raise

# ---------------------------
# Script Entry Point
# ---------------------------
if __name__ == "__main__":
    
    PATHS = {
    "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
    "stores": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
    "sales": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
    "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
}
    
    # Output directory
    OUTDIR = "."
    
    # Run pipeline
    final_forecast = run_forecast_pipeline(PATHS, outdir=OUTDIR, params=PARAMS)

Starting QC Forecast Pipeline...
 Data loaded. Sales: 1,040,191, Claims: 30,836, TODAY: 2024-08-21
Building hotspot ranking...
Generating MVP forecast...
Generating PRO forecast...
Blending and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)

DETAILED REFURBISH INVENTORY ALLOCATION
Target Date: 2024-09-20

STORE-BY-STORE ALLOCATION (Ship by Latest Stock Date):
--------------------------------------------------------------------------------
India - Apple New Delhi
   Ship by: 2024-09-06
   • Apple Watch Series 5                     →  13 units
   • iPhone 11 Pro                            →  13 units
   • Apple One                                →  12 units
   • iPhone 11 Pro Max                        →  12 units
    └─ Store Total: 50 units

Turkey - Apple Ankara
   Ship by: 2024-09-06
   • Apple Watch Series 5                     →  12 units
   • iPad (7th Gen)        